In [1]:
import sys
sys.path.insert(0, '/home/ec2-user/')
import membership_inference_attacks2

# baseline cnn model for fashion mnist
from numpy import mean
from numpy import std
#from matplotlib import pyplot
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import  Lambda
#######from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import confusion_matrix
import random 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import re
from tensorflow.keras import backend as K
#from tensorflow.keras.layers import Activation
from sklearn.neighbors import LocalOutlierFactor
import pandas as pd
from functools import reduce
from collections import Counter
import sys
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix, roc_curve
from membership_inference_attacks2 import black_box_benchmarks
from tensorflow.keras.layers import Dropout
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import copy
from sklearn.metrics import accuracy_score
from random import randint
from keras.regularizers import l2

ALL_XTRAIN, ALL_YTRAIN, ALL_XTEST, ALL_YTEST, ALL_TRAIN_INDEX, ALL_TEST_INDEX =[],[],[],[],[],[]
KNN_OUT, KNN_NOOUT, DIST_OUT, DIST_NOOUT, CLASS_IDX, HIGH_ACTIVATION, LOW_ACTIVATION =[], [], [], [], [], [], []
#EPS=1
#fname="MNIST"
#img_path="C:/Users/senn/Desktop/test/MEMIMG/"
img_path="/home/ec2-user/img/"

Using TensorFlow backend.


In [2]:
# load train and test dataset
def load_dataset_full(dataset, dnn_type=None):

    # load dataset
    
    if dataset=="MNIST":
        (trainX, trainY), (testX, testY) = mnist.load_data() 
        
        dim=28
           
        #concat
        trainX = np.concatenate((trainX, testX))
        trainY = np.concatenate((trainY, testY))
        
        #num of classes
        num_classes=len(np.unique(trainY))
        
    	# reshape dataset to have a single channel
        trainX = trainX.reshape(trainX.shape[0], dim, dim, 1)

    elif dataset=="FMNIST":
        (trainX, trainY), (testX, testY) = fashion_mnist.load_data()
        
        dim=28
                  
        #concat
        trainX = np.concatenate((trainX, testX))
        trainY = np.concatenate((trainY, testY))
        
        #num of classes
        num_classes=len(np.unique(trainY))
        
    	# reshape dataset to have a single channel
        trainX = trainX.reshape(trainX.shape[0], dim, dim, 1)

    elif dataset=="cifar10":
        (trainX, trainY), (testX, testY) = cifar10.load_data()

        dim=32
        #concat
        trainX = np.concatenate((trainX, testX))
        trainY = np.concatenate((trainY, testY))

        #trainX = trainX [0:5000]
        #trainY = trainY [0:5000]
        
        #num of classes
        num_classes=len(np.unique(trainY))
        
    elif dataset=="cifar100":
        (trainX, trainY), (testX, testY) = cifar100.load_data()
 

        dim=32
    
        #concat
        trainX = np.concatenate((trainX, testX))
        trainY = np.concatenate((trainY, testY))
        
        #num of classes
        num_classes=len(np.unique(trainY))

        # reshape dataset to have a single channel
        #trainX = trainX.reshape(trainX.shape[0], pix, pix, 3)


    elif dataset=="purchase100":
        df_tot = pd.read_csv(dataset+'_new.csv', header=None)
        df_tot.dropna(inplace=True)

        trainX = df_tot.iloc[:,1:]
        trainY = df_tot.iloc[:,0]
               
        
        dim=trainX.shape[1]

        
        #num of classes
        num_classes=100

        trainX=np.array(trainX)
        trainY=np.array(trainY)
        
    elif dataset=="texas100":
        df_tot_feats = pd.read_csv(dataset+'_feats.csv', header=None)
        df_tot_feats.dropna(inplace=True)

        df_tot_lab = pd.read_csv(dataset+'_labels.csv', header=None)
        df_tot_lab.dropna(inplace=True)
        
        
        trainX = df_tot_feats
        trainY = df_tot_lab

        dim=trainX.shape[1]

       
        #num of classes
        num_classes=100

        trainX=np.array(trainX)
        trainY=np.array(trainY) 
        
    elif dataset=="location":
        df_tot = pd.read_csv(dataset+'.csv', header=None)
        df_tot.dropna(inplace=True)

     
        trainX = df_tot.iloc[:,1:]
        trainY = df_tot.iloc[:,0]
 
        dim=trainX.shape[1]

       
        #num of classes
        num_classes=30

        trainX=np.array(trainX)
        trainY=np.array(trainY) 
        
    elif dataset=="adult":
        df_tot = pd.read_csv(dataset+'.csv', header=None)
        df_tot.dropna(inplace=True)

        trainX = df_tot.iloc[:,0:df_tot.shape[1]-1]
        trainY = df_tot.iloc[:,-1]

        dim=trainX.shape[1]
    
        #num of classes
        num_classes=2
    
        for j in range(0,trainX.shape[1]):
            if trainX.iloc[:,j].dtypes == object:
                trainX.iloc[:,j]=trainX.iloc[:,j].astype('category')
                trainX.iloc[:,j] = trainX.iloc[:,j].cat.codes
            else:
                sc = StandardScaler()  
                val=np.array(trainX.iloc[:,j]).reshape(-1,1)
                std_data = sc.fit_transform(val)
                std_data = pd.DataFrame(std_data)
                trainX.iloc[:,j]=std_data
                
        trainY = trainY.astype('category')
        trainY = trainY.cat.codes
 
        trainX=np.array(trainX)
        trainY=np.array(trainY)
          
    elif dataset == "synthetic":
        sz=20000
        fin_bin_matrix, bin_class = data_gen(sz)
        
        trainX = fin_bin_matrix
        trainY = bin_class
 
        dim=trainX.shape[1]

       
        #num of classes
        num_classes=2

        trainX=np.array(trainX)
        trainY=np.array(trainY) 
        
 
    # one hot encode target values
    trainY = to_categorical(trainY)


    return trainX, trainY, num_classes, dim

In [3]:
#fin_bin_matrix, bin_class = data_gen(50)

In [4]:
def data_gen(sz):
    size=sz
    half_size = int(size/2)
    bin_matrix = np.mod(np.random.permutation(size*49).reshape(size,49),1)  
    bin_matrix = pd.DataFrame(bin_matrix)
    bin_class = np.zeros(size)
    bin_class[0:half_size] = 1
    for j in range(0,bin_matrix.shape[0]):
        if bin_class[j] == 0:
            s = np.random.normal(0, 0.01, 49)
            bin_matrix.iloc[j] = bin_matrix.iloc[j] + np.abs(s)
        else:
            s = np.random.normal(0, 0.01, 49)
            bin_matrix.iloc[j] = 1 -  np.abs(s)

    bin_class = pd.DataFrame(bin_class)
    #fin_bin_matrix = pd.concat([bin_matrix,bin_class],axis=1)
    return bin_matrix, bin_class

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm


# scale pixels
def prep_pixels_full(train):
    #print(train)
    # convert from integers to floats
    train_norm = train.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    # return normalized images
    return train_norm



def define_attack_model(n_class):
	model = Sequential()

	model.add(Dense(1))
	model.add(Activation("relu"))
    
	model.add(Dense(1))
	model.add(Activation("relu"))

	model.add(Dense(n_class, activation='softmax'))

	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model


# def extract_activation(model,trainX,trainY,train_ix,act_layer):
#     activation_list=[]
#     for i, v in enumerate(model.layers):
#         print(v)
#         s = str(v)
#         a=re.split('\s+', s)
#         if a[0]=='<tensorflow.python.keras.layers.core.Activation':
#             activation_list.append(i)
    
#     global_activation = [K.function([model.input], 
#                               [layer.output])([np.array(trainX), 1])
#                               for layer in model.layers]

    
    
    
#     global_activation = pd.DataFrame(global_activation[act_layer][0])
#     global_activation = global_activation.set_index(train_ix)

#     ####################################################################################################  
#     dist_data=[]
#     pred_mem=[]
#     outlier=[]

#     class_val = np.unique(np.argmax(trainY, axis=1) )
#     trainYY = np.argmax(trainY, axis=1) 
    
#     y=pd.DataFrame(trainYY)
#     rec=pd.DataFrame(train_ix)
#     train_df=pd.concat([rec,y],axis=1)
#     train_df.columns=['rec','y']
    
#     for c_val in class_val:
#        print("\n ",c_val)

#        filter_rec_all = train_df[(train_df['y'] == c_val)]
#        filter_rec_idx = np.array(filter_rec_all['rec'])
       
#        filter_rec=global_activation.loc[filter_rec_idx]
   
#        CLASS_IDX.append(filter_rec_idx)
       
#        m_rec=np.array(np.mean(pd.DataFrame(filter_rec),axis=0))
       
#        dist=np.linalg.norm(filter_rec - m_rec,axis=1)
#        dist_data.append(dist)
       
#        d_mean=np.mean(dist)
       
#        d_mean=np.percentile(dist, 95)
       
       
#        d_dist=pd.DataFrame(dist)       
#        d_dist = d_dist.set_index(filter_rec_idx)
#        d_dist.columns=['dist'] 


       
#        selected_activation = d_dist[(d_dist['dist'] > d_mean)]   
#        activation_idx_to_rec_idx=selected_activation.index.values
       
       
#        #############################################
#        pred_mem.append(activation_idx_to_rec_idx)
#        #############################################
       
#        n_neighbors=10
#        clf = LocalOutlierFactor(n_neighbors=n_neighbors)
#        filter_outlier=clf.fit_predict(filter_rec)
       
#        filter_outlier_df=pd.DataFrame(filter_outlier)
#        filter_outlier_df = filter_outlier_df.set_index(filter_rec_idx)
#        filter_outlier_df.columns=['status'] 

       
#        filter_outlier_df = filter_outlier_df[(filter_outlier_df['status'] == -1)]   
#        outlier_idx_to_rec_idx = filter_outlier_df.index.values
       
#        #np.array(list(selected_outlier_idx['row']))

#        #############################################
#        outlier.append(outlier_idx_to_rec_idx)
#        #############################################
        
    
#     knn_outlier = np.array([item for sublist in outlier for item in sublist])
#     dist_activation = np.array([item for sublist in pred_mem for item in sublist])

#     #all_idx=np.array(list(range(0,len(trainY))))
    
#     no_knn_outlier=(set(train_ix)-set(knn_outlier))
#     no_knn_outlier=np.array(list(no_knn_outlier))

#     no_dist_activation=(set(train_ix)-set(dist_activation))
#     no_dist_activation=np.array(list(no_dist_activation))
    
#     KNN_OUT.append(knn_outlier)
#     KNN_NOOUT.append(no_knn_outlier)
#     DIST_OUT.append(dist_activation)
#     DIST_NOOUT.append(no_dist_activation)

# ####################################################################################################    

#     #global_activation[global_activation>0]=1  
#     values=[]
#     for i in range(0,global_activation.shape[1]):
#         row_values=[]
#         for j in range(0,global_activation.shape[0]):
#             print(i,j)
#             val=1 * (global_activation.iloc[j,i] !=0 ) * ( np.max(global_activation.iloc[j]) -  global_activation.iloc[j,i])
#             row_values.append(val)
#             values.append(row_values)
#     act_values=pd.DataFrame(values).T

#     print(act_values)

#     L_act_pred_mem=[]
#     H_act_pred_mem=[]
    
#     for c_val in class_val:
       
#        filter_rec_all = train_df[(train_df['y'] == c_val)]
#        filter_rec_idx = np.array(filter_rec_all['rec'])
#        filter_rec=global_activation.loc[filter_rec_idx]
 
       
#        m_rec=np.array(np.mean(pd.DataFrame(filter_rec),axis=0))
       
#        dist=np.linalg.norm(filter_rec - m_rec,axis=1)
       
#        d_mean=np.mean(dist)
       
       
#        d_dist=pd.DataFrame(dist)
       
#        print(d_dist)
       
#        d_filter_rec_idx=pd.DataFrame(filter_rec_idx)
#        fin_df=pd.concat([d_filter_rec_idx,d_dist],axis=1)
#        fin_df.columns=['row','dist']
       
#        selected_activation_idx= fin_df[(fin_df['dist'] <= d_mean)]
#        activation_idx_to_rec_idx=np.array(list(selected_activation_idx['row']))
#        L_act_pred_mem.append(activation_idx_to_rec_idx)
       
#        selected_activation_idx= fin_df[(fin_df['dist'] > d_mean)]
#        activation_idx_to_rec_idx=np.array(list(selected_activation_idx['row']))
#        H_act_pred_mem.append(activation_idx_to_rec_idx)
       
#     L_act_pred_mem_idx = np.array([item for sublist in L_act_pred_mem for item in sublist])
#     H_act_pred_mem_idx = np.array([item for sublist in H_act_pred_mem for item in sublist])
    
#     L_act_pred_mem_idx = np.sort(L_act_pred_mem_idx)
#     H_act_pred_mem_idx = np.sort(H_act_pred_mem_idx)
    
    
#     Low_activation_idx=L_act_pred_mem_idx
#     High_activation_idx=H_act_pred_mem_idx


#     #print(Low_activation_idx)
#     #print(High_activation_idx)

#     HIGH_ACTIVATION.append(High_activation_idx)
#     LOW_ACTIVATION.append(Low_activation_idx)




     
# evaluate a model using k-fold cross-validation
def evaluate_model(trainX, trainY, testX, testY, ood_x_test, ood_y_test, dd_x_test, dd_y_test, dataset, model_type, num_classes, dim, channel, init_weights):
    
	
    scores, histories = list(), list()
    TRAIN=[]
    TEST=[]
    pred_status=[]
    
	# define model
    model, act_layer = define_model(dataset, model_type, num_classes, dim, channel)
    #model.set_weights(init_weights)
    
    # fit model
    history = model.fit(trainX, trainY, epochs=EPS, batch_size=32, verbose=0)
    
	# evaluate model
    _, train_acc = model.evaluate(trainX, trainY, verbose=0)
    #print("Train acc : ", (train_acc * 100.0))
    
	# append scores
    #scores.append(test_acc)
    histories.append(history)

    ### train acc classwisw ###
    Y_train_pred = model.predict_classes(trainX, verbose=0)
    Y_train_true = np.argmax(trainY, axis=1) 
    tr_matrix = confusion_matrix(Y_train_true, Y_train_pred)
    tr=tr_matrix.diagonal()/tr_matrix.sum(axis=1)                
    TRAIN.append(tr)
    
    ### test acc classwisw ###
    Y_test_pred = model.predict_classes(testX, verbose=0)
    Y_test_true = np.argmax(testY, axis=1) 
    te_matrix = confusion_matrix(Y_test_true, Y_test_pred)
    te=te_matrix.diagonal()/te_matrix.sum(axis=1)          
    TEST.append(te)
    

    ALL_XTRAIN.append(trainX)
    ALL_YTRAIN.append(trainY)
    ALL_XTEST.append(testX)
    ALL_YTEST.append(testY)

    train_pred=model.predict(trainX, batch_size=32)
    target_train_performance=(train_pred, Y_train_true.astype('int32'))
      
    test_pred=model.predict(testX, batch_size=32)
    target_test_performance=(test_pred, Y_test_true.astype('int32'))
    
    _, ind_test_acc = model.evaluate(testX, testY, verbose=0)
        
        
        
    ood_test_pred = model.predict(ood_x_test, batch_size=32)
    Y_ood_pred_class = model.predict_classes(ood_x_test, verbose=0)
    Y_ood_test_true = np.argmax(ood_y_test, axis=1)
    target_ood_performance=(ood_test_pred, Y_ood_test_true.astype('int32'))
    
    _, ood_test_acc = model.evaluate(ood_x_test, ood_y_test, verbose=0)

    
    
    dd_test_pred=model.predict(dd_x_test, batch_size=32)
    Y_dd_pred_class = model.predict_classes(dd_x_test, verbose=0)
    Y_dd_test_true = np.argmax(dd_y_test, axis=1)
    target_dd_performance=(dd_test_pred, Y_dd_test_true.astype('int32'))
    
    _, dd_test_acc = model.evaluate(dd_x_test, dd_y_test, verbose=0)     
        
    #record prediction status
    
    #train
    tr_pred=(Y_train_true==Y_train_pred)
    pred_status.append(tr_pred)
    
    #test
    te_pred=(Y_test_true==Y_test_pred)
    pred_status.append(te_pred)
    
    #ood
    ood_pred=(Y_ood_test_true==Y_ood_pred_class)
    pred_status.append(ood_pred)
    
    #dd
    dd_pred=(Y_dd_test_true==Y_dd_pred_class)
    pred_status.append(dd_pred)
    
    
    print("Training Acc : ", train_acc, " IND Test Acc : ", ind_test_acc ," OOD Test Acc : ", ood_test_acc, " DD Test Acc : ", dd_test_acc)
    
    return     train_acc, ind_test_acc, histories, TRAIN, TEST , target_test_performance, target_train_performance, model, target_ood_performance, target_dd_performance, pred_status


# evaluate a model using k-fold cross-validation
def evaluate_mod_shadow_model(trainX, trainY, dataset, model_type, num_classes, dim, channel, init_weights):
    
	
    scores, histories = list(), list()
    TRAIN=[]
    TEST=[]
    pred_status=[]
    
	# define model
    model, act_layer = define_model(dataset, model_type, num_classes, dim, channel)
    #model.set_weights(init_weights)
    
    # fit model
    history = model.fit(trainX, trainY, epochs=EPS, batch_size=32, verbose=0)
    
	# evaluate model
    _, train_acc = model.evaluate(trainX, trainY, verbose=0)
   
    train_pred=model.predict(trainX, batch_size=32)
    trainY = np.argmax(trainY,axis=1)
    target_train_performance=(train_pred, trainY.astype('int32'))
      

    
    print("Training Acc : ", train_acc )
    
    return target_train_performance, model



# utility function for showing images
def show_imgs(x_test, decoded_imgs=None, n=10):
    plt.figure(figsize=(10, 5))
    for i in range(n):
        ax = plt.subplot(2, n, i+1)
        item_idx=list(range(0,x_test.shape[0]))
        random.sample(item_idx, n)
        plt.imshow(x_test[i].reshape(28,28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()
    
def classwise_show_imgs(dataset, x_test,y_test, strname, decoded_imgs=None, n=5):
    idx_list=[]
    #sample_per_class=5
      
    class_vals = np.unique(np.argmax(y_test, axis=1) )
    
    trainYY = np.argmax(y_test, axis=1) 
    
    for c_val in class_vals:
        #print(c_val)
        filter_rec_idx= np.where(np.array(trainYY)==c_val)
        if len(filter_rec_idx[0])==0:
            break;
        else:
            sample_per_class=1
            
        
        selected_idx=random.sample(list(filter_rec_idx[0]), sample_per_class)
        idx_list.append(selected_idx)
    
    idx_list = [item for sublist in idx_list for item in sublist]

    n_size=len(idx_list)
    
    plt.figure(figsize=(30, 5))
    
    if dataset in ('MNIST','FMNIST'):
        pix=28
        for i in range(len(idx_list)):
            ax = plt.subplot(2, n_size, i+1)
            image_id=idx_list[i]
            plt.imshow(x_test[image_id].reshape(pix,pix))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
    else:
        pix=32
        for i in range(len(idx_list)):
            ax = plt.subplot(2, n_size, i+1)
            image_id=idx_list[i]
            plt.imshow(x_test[image_id])
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)        


    #plt.savefig(img_path+strname+fname+'.png')

    plt.show()
    
    

def shadow_model(x_shadow, y_shadow, dataset, model_type, num_classes, dim, channel):
    
    
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]
    

    for j in range(0,1):
        
        print("Shadow Model ", j)
        
        x_shadow_train, x_shadow_test, y_shadow_train, y_shadow_test = train_test_split(x_shadow, y_shadow, test_size=0.5)

        model, act_layer = define_model(dataset, model_type, num_classes, dim, channel)
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    	
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))
    	
    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    

    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model



def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
    	print('PPV: 0\nAdvantage: 0')
    else:
    	print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])

                
        
        classifier = define_attack_model(2)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


        
    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall


    
def data_extraction(ind_train_performance,ind_test_performance):

    #test size
    s_test_pred=ind_test_performance[0].shape[0]
    
    #train size
    s_train_pred=ind_train_performance[0].shape[0]
   
    
    #test size <= train size
    if s_test_pred <= s_train_pred:
        
        #randomly pick indices
        indices=list(range(0,s_train_pred))
        rec_idx=random.sample(indices, k = s_test_pred)
        
        trax=ind_train_performance[0][rec_idx]
        tray=ind_train_performance[1][rec_idx]

        tex=ind_test_performance[0]
        tey=ind_test_performance[1]
 
        
    #train size <= test size
    else:
        
        #randomly pick indices
        indices=list(range(0,s_test_pred))
        rec_idx=random.sample(indices, k = s_train_pred)
 
        tex=ind_test_performance[0][rec_idx]
        tey=ind_test_performance[1][rec_idx]

        trax=ind_train_performance[0]
        tray=ind_train_performance[1]
 
    
    test_performance = (tex,tey)
    train_performance = (trax, tray)       
    
    return  train_performance, test_performance 




def indices_extraction(ind_train_performance,ind_test_performance):

    #test size
    s_test_pred=ind_test_performance[0].shape[0]
    
    #train size
    s_train_pred=ind_train_performance[0].shape[0]
   
    
    #test size <= train size
    if s_test_pred <= s_train_pred:
        
        #randomly pick indices
        indices=list(range(0,s_train_pred))
        rec_idx=random.sample(indices, k = s_test_pred)
        
        train_indices = rec_idx
        test_indices = 0
    
    #train size <= test size
    else:
        
        #randomly pick indices
        indices=list(range(0,s_test_pred))
        rec_idx=random.sample(indices, k = s_train_pred)
 
        train_indices = 0
        test_indices = rec_idx
        
    return  train_indices, test_indices 



def _masked_avg(x, mask, axis=0, esp=1e-10):
    return (np.sum(x * mask, axis=axis) / np.maximum(np.sum(mask, axis=axis), esp)).astype(np.float32)
def _masked_dot(x, mask, esp=1e-10):
    x = x.T.astype(np.float32)
    return (np.matmul(x, mask) / np.maximum(np.sum(mask, axis=0, keepdims=True), esp)).astype(np.float32)


def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
   
def memorizable_data(trainX, trainY, b_idx, full_df_size, num_rounds,trainX_full, trainY_full):
#High_memorizable_data, Low_memorizable_data=memorizable_data(trainX, trainY, b_idx, full_df_size,num_rounds,trainX_full, trainY_full)

    import copy
    
    idx_fin_mem=[]
    idx_fin_nmem=[]
    Train_Acc=[]
    Test_Acc=[]
    total_correctness=[]
    Fin_rec_idx=[]
    trainset_mask=[]

    tm_x = copy.deepcopy(trainX)
    tm_y = copy.deepcopy(trainY)
    
    for r in range(1,num_rounds):
       
        print("Run ",r)
                
        df_tr_x = tm_x
        df_tr_y = tm_y

        indices = np.array(list(b_idx))
        
        df_tr_x, df_te_x, df_tr_y, df_te_y,  idx1, idx2  = train_test_split(df_tr_x, df_tr_y, indices, test_size=0.2)
                  
        
        member_index = idx1
        non_member_index = idx2
    
        idx_fin_mem.append(np.array(list(member_index)))
        idx_fin_nmem.append(np.array(list(non_member_index)) )               
    
        
        #members vs nonmember
        subset_idx=idx1
        trainset_mask_t = np.zeros(full_df_size, dtype=np.bool)
        trainset_mask_t[subset_idx] = True
        Fin_rec_idx.append(subset_idx)
            
        
        y_ori=np.argmax(df_tr_y,axis=1)
        test_y_ori=np.argmax(df_te_y,axis=1)
        tot_y_ori=np.argmax(trainY_full,axis=1)
     

        model = define_model(len(np.unique(y_ori)))
	
    
        # fit model
        history = model.fit(df_tr_x, df_tr_y, epochs=EPS, batch_size=32, verbose=0)
        
        pred = model.predict_classes(trainX_full, verbose=0)
        
        class_status=pred==np.array(tot_y_ori)
        total_correctness.append(class_status)
        trainset_mask.append(trainset_mask_t)

 
    trainset_maskx = np.vstack(trainset_mask)
    inv_maskx = np.logical_not(trainset_mask)
    total_correctness = np.vstack(total_correctness)
    #testset_correctnessx = np.vstack(testset_correctness)
                    
    mem_est = _masked_avg(total_correctness, trainset_maskx) - _masked_avg(total_correctness, inv_maskx)
    #infl_est = _masked_dot(testset_correctnessx, trainset_maskx) - _masked_dot(testset_correctnessx, inv_maskx)
    
    
    p = np.percentile(mem_est, 75)
    
    High_memorizable_data=np.where(mem_est>=0.25)
    Low_memorizable_data=np.where(mem_est<0.25)
    
    return High_memorizable_data, Low_memorizable_data,mem_est

def log_loss(a, b):
    SMALL_VALUE = 1e-6
    return [-np.log(max(b[i,a[i]], SMALL_VALUE)) for i in range(len(a))]


def find_ID_and_OOD_data(trainX, trainY, dataset, model_type, num_classes, dim, channel):
    
    init_model, act_layer = define_model(dataset, model_type, num_classes, dim, channel)

    history = init_model.fit(trainX, trainY, epochs=EPS, batch_size=10, verbose=0)
        
    scores_train_loss = init_model.evaluate(trainX, trainY)

    print(init_model.layers[act_layer])
        
    #train_activations = [K.function([init_model.input], 
    #                          [layer.output])([np.array(trainX), 1])
    #                          for layer in init_model.layers]


    layer_output = K.function([init_model.layers[0].input],
                                  [init_model.layers[act_layer].output])
    
    train_activations = layer_output([trainX])[0]

    train_activations= pd.DataFrame(train_activations)
    
    #print(train_activations)
    
    #train_activations= pd.DataFrame(train_activations[act_layer][0])
    y_ori=pd.DataFrame(np.argmax(trainY,axis=1))
    y_ori.columns=['y']      
    train_df=pd.concat([train_activations,y_ori],axis=1)
    
    outliers, inliers=[],[]
    class_val=list(np.unique(np.array(train_df['y'])))
    
    
    for c_val in class_val:
    
        filter_rec_all = train_df[(train_df['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)
        
        clf = LocalOutlierFactor(n_neighbors=5)
        filter_outlier=clf.fit_predict(train_activations.iloc[filter_rec_idx])

        #clf = IsolationForest(n_estimators=10)
        #filter_outlier = clf.fit(train_activations.iloc[filter_rec_idx])
        #filter_outlier    = clf.predict(train_activations.iloc[filter_rec_idx])

        toutliers=np.where(filter_outlier==-1)
        toutliers=filter_rec_idx[toutliers]
        
        tinliers=np.where(filter_outlier==1)
        tinliers=filter_rec_idx[tinliers]
        
        outliers.append(toutliers)
        inliers.append(tinliers)
         
        
    outliers = [item for sublist in outliers for item in sublist]
    inliers = [item for sublist in inliers for item in sublist]

    #outliers=random.sample(outliers,len(outliers))
    #inliers=random.sample(inliers,len(inliers))
    
    return outliers, inliers, train_activations


def extract_prob(train_prob,test_prob):
    train_pred_list=[]
    for ins in range(0, train_prob.shape[0]):
        val= np.argmax(train_prob[ins])
        train_pred_list.append(train_prob[ins][val])
    train_pred_list=np.array(train_pred_list)
    
    
    
    test_pred_list=[]
    for ins in range(0, test_prob.shape[0]):
        val= np.argmax(test_prob[ins])
        test_pred_list.append(test_prob[ins][val])
    
    test_pred_list=np.array(test_pred_list)
    
    return train_pred_list, test_pred_list


def extract_datasets(exp, TRZ, TEZ, SHZ, outliers, inliers):
    #train test shadow  | WO-with outliers NO-no outliers
    print("EXP : ", exp)
    if exp==1:
        print("------------NO NO NO------------")
        
        train=random.sample(set(inliers), TRZ)
        remain_i=set(inliers)-set(train)

        test=random.sample(set(remain_i), TEZ)
        remain_ii=set(remain_i)-set(test)

        shadow=random.sample(set(remain_ii), SHZ)
        
    elif exp==2:
        print("------------NO WO NO------------")
        
        train=random.sample(set(inliers), TRZ)
        remain_i=set(inliers)-set(train)

        shadow=random.sample(set(remain_i), SHZ)

        test=random.sample(set(outliers), TEZ)
        

    elif exp==3:
        print("------------NS NS NS------------")
        all_data=np.concatenate((outliers,inliers))
        
        train=random.sample(set(all_data), TRZ)
        remain_i=set(all_data)-set(train)

        test=random.sample(set(remain_i), TEZ)
        remain_i=set(remain_i)-set(test)
        
        shadow=random.sample(set(remain_i), SHZ)
        
    return train, test, shadow    

def sample_selection(trainX_full,trainY_full,k_val,tr_idx):
   
    class_val = np.unique(np.argmax(trainY_full, axis=1) )

    idxY = pd.DataFrame(tr_idx)
    trainY = pd.DataFrame(np.argmax(trainY_full, axis=1) )
    trainYY=pd.concat([idxY,trainY],axis=1)
    trainYY.columns=['idx','y']
        
    
    balance_filter_rec_idx_list=[]
    

    for c_val in class_val:
        print(c_val)

        filter_rec_idx = trainYY[(trainYY['y'] == c_val)]
        filter_rec_idx =filter_rec_idx['idx']
       
        
        filter_rec_idx=random.sample(list(filter_rec_idx), k = k_val)       
        balance_filter_rec_idx_list.append(filter_rec_idx)
        
    return balance_filter_rec_idx_list

#import cv2
#import numpy as np

def dd_noisy_img_creation(dd_X):
    dd_img=[]
    for j in dd_X:
        img =j
        gauss = np.random.normal(0,3,img.size)
        gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
        noise_img = img + img * gauss
        dd_img.append(noise_img)
    return dd_img

def dd_noisy_img_creation_28pix(dd_X):
    dd_img=[]
    for j in dd_X:
        img =j
        #gauss = np.random.normal(8,1,img.size)
        gauss = np.random.randn(8,1,img.size)
        gauss = gauss.reshape(img.shape[0],img.shape[1],1).astype('uint8')
        noise_img = img + img * gauss
        dd_img.append(noise_img)
    return dd_img


def add_salt_pepper_noise (image):
  prob = 0.2  
  output=np.zeros (image.shape, np.uint8)
  thres=1-prob
  for i in range (image.shape [0]):
    for j in range (image.shape [1]):
      rdn=random.random ()
      if rdn<prob:
        output [i] [j]=0
      elif rdn>thres:
        output [i] [j]=255
      else:
        output [i] [j]=image [i] [j]
  return output
  

def dd_noisy_img_creation_sandp(dd_X):
    dd_img=[]
    for j in dd_X:
        img =j
        salt_pepper_noise_imgs = add_salt_pepper_noise(img)
        dd_img.append(salt_pepper_noise_imgs)
    return dd_img

def dd_noisy_bin_data_creation(dd_X):
    dd_bin=[]
    for j in range(0,dd_X.shape[0]):
        
        selected_idx = random.sample(list(range(0,dd_X.shape[1])), int( dd_X.shape[1]*0.8 ) )
        #print(selected_idx)
        for i in selected_idx:
            #print(i)
            #print(j[i])
            if dd_X[j,i] == 0 :
                dd_X[j,i] = 1 #np.random.choice(5, 1)[0]#1
            else:
                dd_X[j,i] = 0

    return dd_X

def dd_noisy_bin_data_binnoise(dd_X,noise=0.9):
    dd_bin=[]
    for j in range(0,dd_X.shape[0]):
        dd_bin.append(np.random.binomial(1,noise, dd_X.shape[1]))
    return np.array(dd_bin)

def dd_noisy_bin_data(dd_X,noise):
    dd_bin=[]
    p=noise
    dd_X_tmp = copy.deepcopy(dd_X) 
    for j in range(0,dd_X_tmp.shape[0]):
        tmp_dd_X_tmp=dd_X_tmp[j]
        for i, value in enumerate(tmp_dd_X_tmp):
            v=np.random.choice([value,np.abs(value-1)], p=[1-p,p])
            dd_X_tmp[j][i] = 5
    return dd_X_tmp
        



def fake_members(true_x, true_y, pred_y, classifier, max_t):
    true_y_t = np.argmax(true_y,axis=1)
    per_instance_loss = np.array(log_loss(true_y_t,pred_y))
    #print (per_instance_loss)
    
    counts = np.zeros(len(true_x))
    
    for t in range(max_t):
        noisy_x = true_x + generate_noise(1)
                    
        pred_y = classifier.predict(noisy_x, batch_size=32)

        noisy_per_instance_loss = np.array(log_loss(true_y_t, pred_y))
        
        counts += np.where(noisy_per_instance_loss > per_instance_loss, 1, 0)
    return counts, per_instance_loss


def loss_thre_setting(tr_values, te_values, train_class, test_class):
    value_list = np.concatenate((tr_values, te_values))
    class_vals = np.unique(np.append(train_class, test_class))
    thresh = dict()
    #print(class_vals)
    for clz in class_vals:
        print(clz)
        train_ind = np.where(train_class == clz)
        test_ind = np.where(test_class == clz)
        #value_sd = np.std(value_list)
        thre, max_acc = 0, 0
        value_list_tmp = np.concatenate((tr_values[train_ind], te_values[test_ind]))
        for value in value_list_tmp:
            tr_ratio = np.sum(tr_values>=value)/(len(tr_values)+0.0)
            te_ratio = np.sum(te_values<value)/(len(te_values)+0.0)
            acc = 0.5*(tr_ratio + te_ratio)
            if acc > max_acc:
                thre, max_acc = value, acc
        thresh[clz] = thre
    return thresh   
    





In [5]:
def shadow_model_attack(x_shadow, y_shadow, dataset, model_type, num_classes, dim, channel):
    
    
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]
    

    for j in range(0,1):
        
        print("Shadow Model ", j)
        
        x_shadow_train, x_shadow_test, y_shadow_train, y_shadow_test = train_test_split(x_shadow, y_shadow, test_size=0.5)

        model, act_layer = define_model(dataset, model_type, num_classes, dim, channel)
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    	
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))
    	
    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    

    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model


In [6]:
def shadow_model_noisy (x_shadow, y_shadow, dataset, model_type, num_classes, dim, channel):
    
    
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]
    

    for j in range(0,1):
        
        print("Shadow Model ", j)
        
        x_shadow_train, x_shadow_test, y_shadow_train, y_shadow_test = train_test_split(x_shadow, y_shadow, test_size=0.5)

        #get noisy shadow data
        if dataset in ('MNIST', 'FMNIST', 'CIFAR10'):

            mean = 0.0   # some constant
            sd = 0.1    # some constant (standard deviation)
            noisy_img = x_shadow_test + np.random.normal(mean, sd, x_shadow_test.shape)
            x_shadow_test = np.clip(noisy_img, 0, 255)

            #distance to original
            #dist_list=np.array(distf(true_x, noisy_x,"img"))
       
            
        else:
            sd = 0.3
            x_shadow_test = dd_noisy_bin_data(x_shadow_test,sd)
          
            #distance to original
            #dist_list=np.array(distf(true_x, noisy_x,"-"))
        
        
        model, act_layer = define_model(dataset, model_type, num_classes, dim, channel)
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    	
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))
    	
    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    

    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model



In [7]:
############# Merlin attack ##################################
import scipy.spatial.distance as dist
from scipy.spatial import distance_matrix


def merlin_attack (x, y, x_test, y_test, train_prob, test_prob, model, sd, datatype, max_t):
    
    precision_merlin, recall_merlin, acc_merlin = [], [], []
    
    train_counts, train_per_instance_loss, tr_dist_ic_list, tr_status = loss_increase_counts(x, y, train_prob[0], model, sd, datatype, max_t)
    test_counts, test_per_instance_loss, te_dist_ic_list, te_status  = loss_increase_counts(x_test, y_test, test_prob[0],model, sd, datatype, max_t)

    merlin_thresh = thre_setting(train_counts, test_counts)
        
    f_train_dist = [item for sublist in  tr_dist_ic_list for item in sublist]
    f_test_dist  = [item for sublist in te_dist_ic_list for item in sublist]

    ba_thresh = thre_setting(f_train_dist,f_test_dist)

    #ba_thresh = mod_thre_setting (f_train_dist,f_test_dist)
    
    #print(tr_dist_ic_list)
    
    dist_list= (tr_dist_ic_list,te_dist_ic_list)
    
    
    return merlin_thresh, train_counts, test_counts, ba_thresh, tr_dist_ic_list, te_dist_ic_list, tr_status, te_status  

    
def merlin_exe (merlin_thresh,train_counts,test_counts):
    #IND data 
    train_merlin_members =  np.where(train_counts >= merlin_thresh, 1, 0)
    test_merlin_members =  np.where(test_counts >= merlin_thresh, 1, 0)

    #train vs test
    tp = np.sum(train_merlin_members==1)
    fp = np.sum(test_merlin_members==1)
    tn = np.sum(test_merlin_members==0)
    fn = np.sum(train_merlin_members==0)

    print("TP : ",tp)
    print("TN : ",tn)
    print("FN : ",fn)
    print("FP : ",fp)


    precision = (tp/(tp+fp))
    recall = (tp/(tp+fn))
    acc = (tp+tn)/(tp+tn+fn+fp)

    print("Precision : ", (tp/(tp+fp)))
    print("Recall : ", (tp/(tp+fn)))
    print("Acc : ", acc)
    
    return precision, recall, acc





def log_loss(a, b):
    SMALL_VALUE = 1e-6
    return [-np.log(max(b[i,a[i]], SMALL_VALUE)) for i in range(len(a))]

def generate_noise(shape,sd):
    mu, sigma = 0, sd # mean and standard deviation
    
    noise = np.random.normal(mu, sigma, shape)
   
    return noise  
    
def loss_increase_counts(true_x, true_y, pred_y, classifier, sd, datatype, max_t):
    
    true_y_t = np.argmax(true_y,axis=1)
    
    per_instance_loss = np.array(log_loss(true_y_t,pred_y))
   
    counts = np.zeros(len(true_x))
    
    mean_dist_cc, mean_dist_ic, dist_ic_list, mem_idx, nm_idx, membership, class_status  = [], [], [], [], [], [], []

    ori_y = classifier.predict(true_x, batch_size=32)   
    correct_ori_pred = np.where(np.argmax(ori_y,axis=1)==true_y_t)[0]
    incorrect_ori_pred = np.where(np.argmax(ori_y,axis=1)!=true_y_t)[0]
    
    for t in range(max_t):
        
        if dataset in ('MNIST', 'FMNIST', 'CIFAR10'):

            mean = 0.0   # some constant
            #std = sd    # some constant (standard deviation)
            noisy_img = true_x + np.random.normal(mean, sd, true_x.shape)
            noisy_x = np.clip(noisy_img, 0, 255)

            #distance to original
            dist_list=np.array(distf(true_x, noisy_x,"img"))
       
            
        else:
            noisy_x = dd_noisy_bin_data(true_x,sd)
          
            #distance to original
            dist_list=np.array(distf(true_x, noisy_x,"-"))
       
        #processing for merlin attack
        pred_y = classifier.predict(noisy_x, batch_size=32)
        
        noisy_per_instance_loss = np.array(log_loss(true_y_t, pred_y))
        
        counts += np.where(noisy_per_instance_loss > per_instance_loss, 1, 0)
        
        
        #processing for boundary attack
        
        noisy_y = classifier.predict(noisy_x, batch_size=32)   
        correct_noisy_idx = np.where(np.argmax(noisy_y,axis=1)==true_y_t)[0]
        incorrect_noisy_idx = np.where(np.argmax(noisy_y,axis=1)!=true_y_t)[0]
          
        
        
        if datatype == "shadow":

            if len(incorrect_ori_pred)>0:
                dist_list[incorrect_ori_pred]=0    
                        
            #get only the ones with incorrect classifications - dist should be higher for training low for test        
            dist_ic_list.append(dist_list[incorrect_noisy_idx])
            #print("incorrect_noisy_idx : ", len(incorrect_noisy_idx))
            
 
            
        else:
            
            #if len(incorrect_ori_pred)>0:
            #    dist_list[incorrect_ori_pred]=0    
  
            #dist_ic_list.append(dist_list[incorrect_noisy_idx])
            #("incorrect_noisy_idx : ", len(incorrect_noisy_idx))

            dist_ic_list.append(dist_list)  
            
            
            pred_y = classifier.predict(noisy_x, batch_size=32)   
            status = np.where(np.argmax(pred_y,axis=1)==true_y_t,1,0)
            class_status.append(status)
 
            
 
    return counts, per_instance_loss, dist_ic_list,class_status    
    
def get_inference_threshold(pred_vector, true_vector, fpr_threshold=None):
    fpr, tpr, thresholds = roc_curve(true_vector, pred_vector, pos_label=1)
        # return inference threshold corresponding to maximum advantage
    if fpr_threshold == None:
        return thresholds[np.argmax(tpr-fpr)]
        # return inference threshold corresponding to fpr_threshold
    for a, b in zip(fpr, thresholds):
        if a > fpr_threshold:
            break
        alpha_thresh = b
    return alpha_thresh    


def thre_setting(tr_values, te_values):
    value_list = np.concatenate((tr_values, te_values))
    value_sd = np.std(value_list)
    thre, max_acc = 0, 0
    for value in value_list:
        tr_ratio = np.sum(tr_values>=value)/(len(tr_values)+0.0)
        te_ratio = np.sum(te_values<value)/(len(te_values)+0.0)
        acc = 0.5*(tr_ratio + te_ratio)
        if acc > max_acc:
            thre, max_acc = value, acc
            #thre=thre+np.choice()
    return thre     



def mod_thre_setting(f_train_dist,f_test_dist):
    
    num_increments = 100
    
    tau_increment = np.amax([np.amax(f_train_dist), np.amax(f_test_dist)]) / num_increments

    acc_max = 0.0
    
    distance_threshold_tau = 0.0
    
    for i_tau in range(1, num_increments):
        is_member_train = np.where(f_train_dist > i_tau * tau_increment, 1, 0)
        
        is_member_test = np.where(f_test_dist > i_tau * tau_increment, 1, 0)
        
        
        acc = (np.sum(is_member_train) + (is_member_test.shape[0] - np.sum(is_member_test))) / (
                is_member_train.shape[0] + is_member_test.shape[0]
            )
    
        if acc > acc_max:
            distance_threshold_tau = i_tau * tau_increment
            acc_max = acc

    return distance_threshold_tau
    
    



def distf(true_x, noisy_x,dt):
    dist_arr=[]
    for j in range(true_x.shape[0]):
        if dt=="img":
            d = dist.euclidean(true_x[j].flatten(), noisy_x[j].flatten()) 
        else:
            d = np.sum(true_x[j]!=noisy_x[j])/true_x[j].shape[0]
                
        dist_arr.append(d)
    return dist_arr

In [8]:
def boundary_attack_results (true_membership, pred_membership_ba):
    #true_membership = np.concatenate([np.ones(x_train[:num_rec].shape[0]), np.zeros(x_test[:num_rec].shape[0])])
    #pred_membership_ba = np.append(np.array(train_membership),np.array(test_membership))


    tn, fp, fn, tp = confusion_matrix(true_membership, pred_membership_ba).ravel()

    print("TP : ", tp )
    print("TN : ", tn)
    print("FN : ", fn )
    print("FP : ", fp,"\n")                                  

    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    acc=(tp+tn)/(tp+tn+fn+fp)

    print("precision : ",precision)
    print("recall : ",recall)
    print("acc : ", acc)                                  
    print("\n\n")

    return precision, recall, acc
    
def boundary_attack_exe (ba_thresh, dist, status):
    
    nm_idx, mem_idx =[], [] 
    
    members, nonmembers =  [], []

    for i in range(0,len(dist)):
        #print(i)
        #incorrecly classified, dist higher than t -> members
        #print( status[i]==0))
        ICC_MEM = np.where( ( status[i]==0) & (dist[i] > ba_thresh) )[0]
        members.append(ICC_MEM)
        
        #incorrecly classified, dist less than t -> nonmembers
        ICC_NMEM = np.where( ( status[i]==0) & (dist[i] < ba_thresh) )[0]
        nonmembers.append(ICC_NMEM)
        
        #correcly classified, dist higher than t -> members
        CC_MEM = np.where( ( status[i]==1) & (dist[i] > ba_thresh) )[0]
        members.append(CC_MEM)
        
        #correcly classified, dist less than t -> could  be members or nonmembers -> approx as members
        CC_MN = np.where( ( status[i]==1) & (dist[i] < ba_thresh) )[0]        
        nonmembers.append(CC_MN)
       
    nm_flat_list = [item for sublist in nonmembers for item in sublist]
    mem_flat_list = [item for sublist in members for item in sublist]
    
    
    nm_counter=Counter(nm_flat_list)
    mm_counter=Counter(mem_flat_list)

    tmp_key=list(nm_counter.keys()),list(mm_counter.keys())
    all_keys = np.unique([item for sublist in tmp_key for item in sublist])
    
    #all_keys = list(np.unique(nm_counter.keys(),mm_counter.keys())[0][0])        
        

    membership = []
    for key in all_keys:
        cnt_nm = nm_counter[key]
        cnt_mm = mm_counter[key]

        if cnt_nm >= cnt_mm:
            membership.append(0)
        else:
            membership.append(1)
                
    return np.array(membership)
    

In [9]:
def loss_mia(loss_tresh_list, shadow_train_instance_loss, shadow_test_instance_loss, train_class,test_class):
    all_classes = np.append(train_class,test_class)
    unique_classes = np.unique(all_classes)
    fn, tp, fp, tn = 0, 0, 0, 0
    
    loss_outlier=[]
    loss_inliers=[]
    
    
    for clz in unique_classes:
        train_ind = np.where(train_class == clz)
        test_ind = np.where(test_class == clz)
        thre = loss_tresh_list[clz]              
        
        fn += np.sum(shadow_train_instance_loss[train_ind]>=thre)
        tp += np.sum(shadow_train_instance_loss[train_ind]<thre)
            
        fp += np.sum(shadow_test_instance_loss[test_ind]<thre)
        tn += np.sum(shadow_test_instance_loss[test_ind]>=thre)
        
        #loss_inliers.append( np.where( t_tshadow_train_instance_loss[train_ind] >= thre))
        #loss_outlier.append( np.where( t_te_values[self.t_te_labels==num] >= thre, 1,0))
 
    
 
    print("TP : ", tp )
    print("TN : ", tn)
    print("FN : ", fn )
    print("FP : ", fp)

        
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
        
    acc=(tp+tn)/(tp+tn+fn+fp)

    print("precision : ",precision)
    print("recall : ",recall)
    print("acc : ", acc)
    print("\n\n")
    
    return precision, recall, acc

def extract_activation(model,trainX,trainY,act_layer):
    activation_list=[]
 
    print("Layer list : ")
    print(model.layers)
    
    global_activation = [K.function([model.input], 
                              [layer.output])([np.array(trainX), 1])
                              for layer in model.layers]

    
    #print(pd.DataFrame(global_activation[act_layer][0]))
    #print(pd.DataFrame(global_activation[act_layer]))
    
    global_Df = pd.DataFrame(global_activation[act_layer][0])
    

    
    train_ix = list(range(trainY.shape[0]))
    global_Df['y'] = np.argmax(trainY,axis=1)
    class_val = np.unique(np.argmax(trainY,axis=1))
    train_df = global_Df
    
    return train_df



from sklearn.decomposition import PCA

def activation_mia(act_df_full,class_val,act_layer,nn):
    
    CLASS_IDX, dist_data, pred_mem, outlier, inlier = [], [], [], [], []

    #print(act_df_full.head())
    
    for c_val in class_val:

        filter_rec_all = act_df_full[(act_df_full['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)

        
        filter_rec=act_df_full.loc[filter_rec_idx]
        filter_rec=filter_rec.loc[:, filter_rec.columns != 'y']

        #print(c_val)

        clf = LocalOutlierFactor(n_neighbors=nn)
        filter_outlier=clf.fit_predict(filter_rec)

        filter_outlier_df = pd.DataFrame(filter_outlier)
        filter_outlier_df = filter_outlier_df.set_index(filter_rec_idx)
        filter_outlier_df.columns=['status'] 

        
        
        #print(filter_outlier_df.loc[filter_outlier_df['status'] != -1]  )
        #print(filter_outlier_df.loc[filter_outlier_df['status'] == 1]  )
        
        outlier_df = filter_outlier_df.loc[filter_outlier_df['status'] == -1]
        outlier_idx_to_rec_idx = outlier_df.index.values
        #print(outlier_idx_to_rec_idx)

        inlier_df = filter_outlier_df.loc[filter_outlier_df['status'] == 1]    
        #print(filter_inlier_df)
        inlier_idx_to_rec_idx = inlier_df.index.values
        #print(inlier_idx_to_rec_idx)

        outlier.append(outlier_idx_to_rec_idx)
        inlier.append(inlier_idx_to_rec_idx)
        

    nonmembers = np.array([item for sublist in outlier for item in sublist])
    members = np.array([item for sublist in inlier for item in sublist])
        
    members=np.array(list(members))
    nonmembers=np.array(list(nonmembers))
    
    #print("members : ", inlier)
    #print("nonmembers : ", len(nonmembers))
    
    pred_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
    #print(len(nonmembers))
    
    return pred_membership
    
    
def prep_attack_data(n_attack_data):

    attack_mem = pd.DataFrame(n_attack_data[0][0])
    attack_nmem = pd.DataFrame(n_attack_data[0][1])
    
    attack_mem_status = pd.DataFrame(n_attack_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(n_attack_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(n_attack_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(n_attack_data[2][1])
    real_class_nmem.columns = ["y"]

    memdf = pd.concat([attack_mem,attack_nmem],axis=0)
    memdf = memdf.reset_index(drop=True)

    memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
    memstatus = memstatus.reset_index(drop=True)

    realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
    realclass = realclass.reset_index(drop=True)

    attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return attack_df


def shokri_attack(attack_df, mem_validation, nmem_validation):
    
    predicted_membership, predicted_nmembership, true_membership, TP_idx, TN_idx  = [], [], [], [], []

    class_val = np.unique(attack_df['y'])
    ncval=attack_df.shape[1]-1
    
    for c_val in class_val:

        print(c_val)
        
        filter_rec_all = attack_df[(attack_df['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)
        
        attack_feat = filter_rec_all.iloc[:, 0:ncval]
        attack_class = filter_rec_all['membership']
             
        d=1
        pix = attack_feat.shape[1]
        
        attack_model, _ = attack_mlp(pix,d)
        
       
        history = attack_model.fit(attack_feat, attack_class, epochs=EPS, batch_size=32, verbose=0)
        
        mcval=mem_validation.shape[1]-1
        
        
        check_mem_feat = mem_validation[mem_validation['y']==c_val]
        check_nmem_feat = nmem_validation[nmem_validation['y']==c_val]
        
        if (len(check_mem_feat)!=0) and (len(check_nmem_feat)!=0):
        
            check_mem_feat_idx =  np.array(check_mem_feat.index)


            check_nmem_feat_idx =  np.array(check_nmem_feat.index)

            #print(check_nmem_feat_idx)
            #print(np.argmax(mpred,axis=1)==0)


            mpred = attack_model.predict(np.array(check_mem_feat))    
            predicted_membership.append(np.argmax(mpred,axis=1) )

            nmpred = attack_model.predict(np.array(check_nmem_feat))    
            predicted_nmembership.append(np.argmax(nmpred,axis=1) )        



            TP_idx.append(check_mem_feat_idx[np.where(np.argmax(mpred,axis=1)==1)[0]])

            TN_idx.append(check_nmem_feat_idx[np.where(np.argmax(nmpred,axis=1)==0)[0]])

    pred_members = np.array([item for sublist in predicted_membership for item in sublist])
    pred_nonmembers = np.array([item for sublist in predicted_nmembership for item in sublist])
    
    TP_idx_list = np.array([item for sublist in TP_idx for item in sublist])
    TN_idx_list = np.array([item for sublist in TN_idx for item in sublist])
    
    members=np.array(list(pred_members))
    nonmembers=np.array(list(pred_nonmembers))
    
    pred_membership = np.concatenate([members,nonmembers])
    ori_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
    return pred_membership, ori_membership, TP_idx_list, TN_idx_list

In [10]:
def define_model(dataset , model_type, n_class, dim, channel):
    if dataset in ('cifar10', 'cifar100') and model_type=="Alexanet":
        
        model, act_layer = build_alexanet(n_class,dim,channel) 
    
    elif dataset in ('cifar10', 'cifar100') and model_type=="VGG16":
         
        model, act_layer = build_vgg (n_class,dim,channel)
    
    elif dataset=="purchase100" and model_type=="DNN":
        
        model, act_layer = buid_purchase_dnn(n_class,dim)
    
    
    elif dataset=="texas100" and model_type=="DNN":   
       
        model, act_layer = buid_texas_dnn(n_class,dim)
    
    elif dataset=="location" and model_type=="DNN":   
       
        model, act_layer = buid_location_dnn(n_class,dim)

    elif dataset in ('MNIST','FMNIST') and model_type=="Alexanet": #MLP
         
        model, act_layer = build_simple_cnn (n_class,dim,channel)
        #model, act_layer = build_alexanet(n_class,dim,channel) 

    elif dataset in ('adult') and model_type=="MLP":
         
        model, act_layer = build_simple_mlp (n_class,dim,channel)
                    
    elif dataset in ('synthetic') and model_type=="MLP":
         
        model, act_layer = build_simple_mlp (n_class,dim,channel)
     
    return model, act_layer

In [11]:
def buid_purchase_dnn(n_class,dim):
    model = Sequential()
    
    model.add(Dense(600, input_dim=dim))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    #model.add(Dense(1024), kernel_regularizer=l2(0.001))
    #model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(512, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(256, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(128, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    #opt = SGD(lr=0.01, momentum=0.9)
    #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer
    

In [12]:
def buid_texas_dnn(n_class,dim):
    model = Sequential()
    
    model.add(Dense(1024, input_dim=dim, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(512, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(128, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
       
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer
    

In [13]:

def buid_location_dnn(n_class,dim):
    model = Sequential()
    
    model.add(Dense(512, input_dim=dim, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(248, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(128, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(64, kernel_regularizer=l2(0.0007)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
       
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer
    

In [14]:
def build_simple_cnn(n_class,pix,d):

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(pix, pix, d)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(32,  kernel_initializer='he_uniform'))
    model.add(Activation("relu"))
    model.add(Dense(n_class, activation='softmax'))
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


    act_layer=3
    
    return model, act_layer

In [15]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [16]:
def attack_mlp(pix,d):

    model = Sequential()
    model.add(Dense(64, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.1))

#     model.add(Dense(32))
#     model.add(Activation("tanh"))
#     model.add(Dropout(0.01))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=1
    
    return model, act_layer

In [17]:
#model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(1,28,28), data_format='channels_first'))
# AlexNet model
class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'same', activation= 'relu',
                        input_shape= input_shape, 
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'same', data_format= None))

        self.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'same',  data_format= None)) 

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'same', data_format= None))

        self.add(Flatten())
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(1000, activation= 'relu'))
        self.add(Dense(num_classes, activation= 'softmax'))

        self.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        

In [18]:
def build_alexanet (n_class,pix,d):
    
        
    model = AlexNet((pix, pix, d), n_class)
    
    act_layer=11

    #model.summary()
    
    return model, act_layer

In [19]:
####load full dataset#### ****
dataset='FMNIST' #'MNIST', 'cifar10', 'cifar100', 'purchase100', 'texas100' 'FMNIST' #location #synthetic
model_type='Alexanet'  #Alexanet #VGG16 #DNN #CNN #MLP
trainX_full, trainY_full, num_classes, dim = load_dataset_full(dataset,model_type) 
 
if dataset in ('MNIST', 'FMNIST', 'cifar10', 'cifar100'):
    trainX_full = prep_pixels_full(trainX_full)


#####unbalanced classes########
trainX=trainX_full
trainY=trainY_full


####load completely different dataet####
if dataset == 'cifar10':
    dd_dataset='cifar10'
    per_class_sample=2000
    channel=3
    EPS=200
    act_layer=11
    
elif dataset == 'MNIST':
    dd_dataset='FMNIST'
    per_class_sample=2000
    channel=1
    EPS=10#0
    act_layer=4
    
elif dataset == 'cifar100':
    dd_dataset='cifar100'
    per_class_sample=100
    channel=3
    EPS=100
    act_layer=11
    
elif dataset == 'purchase100':
    dd_dataset='purchase100'
    per_class_sample=150
    channel=0
    EPS=50
    act_layer=6
    
elif dataset == 'texas100':
    dd_dataset='texas100'
    per_class_sample=100
    channel=0    
    EPS=8
    num_classes=100
    act_layer=6
    
elif dataset == 'adult':
    dd_dataset='adult'
    per_class_sample=5000
    channel=0   
    EPS=200
    act_layer=3
    
elif dataset == 'FMNIST':
    dd_dataset='MNIST'
    per_class_sample=2000
    channel=1 
    EPS=100
    act_layer=4
    
elif dataset == 'location':
    dd_dataset='location'
    per_class_sample=40
    channel=1 
    EPS=100
    act_layer=6
    
elif dataset == 'synthetic':
    dd_dataset='synthetic'
    per_class_sample=2500
    channel=1 
    EPS=10
    
init_model,_= define_model(dataset , model_type, num_classes, dim, channel)
init_weights = init_model.get_weights()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [20]:
TRZ=per_class_sample*num_classes
OUTZ=TRZ
SHZ=int(TRZ*0.5)
TEZ=OUTZ



itp, ifp, ifn, itn, iprecision, iadvj, iacc, irecall = [], [], [], [], [], [], [], []

me_iprecision, me_iacc, me_recall, me_train_acc, me_test_acc = [], [], [], [], []
e_iprecision, e_iacc, e_recall, e_train_acc, e_test_acc = [], [], [], [], []
conf_iprecision, conf_iacc, conf_recall, conf_train_acc, conf_test_acc = [], [], [], [], []
cc_precision, cc_recall, cc_acc = [],[],[]


i_train_acc=[]
i_test_acc=[]


In [21]:
#########################Normal training ##################################

#######training data#######
IND_x_train=trainX
IND_y_train=trainY
               
               
##########get training data from inliers with balanced classes##########
if dataset not in ['purchase100']:
    tr_idx=list(range(0,IND_x_train.shape[0]))
    idx = sample_selection(IND_x_train,IND_y_train,per_class_sample,tr_idx)
    idx = random.sample(idx,len(idx))
    idx = np.array([item for sublist in idx for item in sublist])
    
else:
    #####un-balanced classes########
    tot_class_samples = 20000
    tr_idx=list(range(0,IND_x_train.shape[0]))
    idx = random.sample(tr_idx,tot_class_samples)

    TRZ=tot_class_samples
    OUTZ=TRZ
    SHZ=int(TRZ*0.5)
    TEZ=OUTZ

    
x_train=trainX[idx]
y_train=trainY[idx]               

t_size = x_train.shape[0]

#####  shuffel data  #########
x_train, y_train = shuffle(x_train, y_train)

remaining_inliers=set(tr_idx)-set(idx)
           
##########get testing data from inliers##########               

test_idx=random.sample(set(remaining_inliers), TEZ)
remaining_inliers_ii=set(remaining_inliers)-set(test_idx)      

x_test=trainX[test_idx]
y_test=trainY[test_idx]

##########get shadow data from inliers##########               
shadow_idx=random.sample(set(remaining_inliers_ii), SHZ)

x_shadow=trainX[shadow_idx]
y_shadow=trainY[shadow_idx]

#########for creating noisy data ###############
remaining_inliers_iii=set(remaining_inliers_ii)-set(shadow_idx)      
noisy_idx = remaining_inliers_iii


##########get ood test data from inliers##########               

#ood_x_test=trainX[outliers]
#ood_y_test=trainY[outliers]
    




0
1
2
3
4
5
6
7
8
9


# 

In [22]:
##########get dd data ##########               
import random

random.seed(10)

noise = 1

if dd_dataset in ('MNIST', 'FMNIST'):
    trainX_full_cood, trainY_full_cood, num_classes_cood, dimentionality_cood = load_dataset_full(dd_dataset)
    dd_x_test=trainX_full_cood
    dd_y_test=trainY_full_cood 

    #dd_x_test=trainX[list(noisy_idx)]
    #dd_y_test=trainY[list(noisy_idx)]

    #dd_x_test = dd_noisy_img_creation_28pix(dd_x_test)
    
    dd_x_test = dd_noisy_img_creation_sandp(dd_x_test)
    dd_x_test = np.array(dd_x_test)

    
elif dd_dataset in ('cifar10', 'cifar100'):
    dd_x_test=trainX[list(noisy_idx)]
    dd_y_test=trainY[list(noisy_idx)]

    dd_x_test = dd_noisy_img_creation(dd_x_test)
    dd_x_test = np.array(dd_x_test)
    
elif dd_dataset in ('purchase100'):
    noisy_idx = list(noisy_idx)[0:20000]
    dd_x_test=trainX[list(noisy_idx)]
    dd_y_test=trainY[list(noisy_idx)]

    #dd_x_test = dd_noisy_bin_data_binnoise(dd_x_test)
    dd_x_test = dd_noisy_bin_data(dd_x_test,noise)
    
elif dd_dataset in ('texas100'):
    noisy_idx = list(noisy_idx)[0:10000]
    dd_x_test=trainX[list(noisy_idx)]
    dd_y_test=trainY[list(noisy_idx)]

    #dd_x_test = dd_noisy_bin_data_binnoise(dd_x_test)
    dd_x_test = dd_noisy_bin_data(dd_x_test,noise)
    
elif dd_dataset in ('location'):
    dd_x_test=trainX[list(noisy_idx)]
    dd_y_test=trainY[list(noisy_idx)]

    #dd_x_test = dd_noisy_bin_data_binnoise(dd_x_test)
    dd_x_test = dd_noisy_bin_data(dd_x_test,noise)
    
elif dd_dataset in ('adult'):
#     dd_x_test=trainX[list(noisy_idx)]
#     dd_y_test=trainY[list(noisy_idx)]

#     dd_x_test = dd_noisy_bin_data(dd_x_test,noise)
    
    
    trainX_full_cood, _, _, _ = load_dataset_full('purchase100')
    trainX_full_cood = pd.DataFrame(trainX_full_cood)
    
    expected_dim = trainX_full.shape[1]
    expected_instances = trainX_full_cood.shape[0]

    dd_x_test = trainX_full_cood.iloc[:,1:15]
    dd_x_test = np.array(dd_x_test)
    dd_y_test = [randint(0, 1) for p in range(0, expected_instances)]
    dd_y_test = np.array(dd_y_test)
    dd_y_test = to_categorical(dd_y_test)


    #dd_x_test=trainX[list(noisy_idx)]
    #dd_y_test=trainY[list(noisy_idx)]
    
    dd_x_test = pd.DataFrame(dd_x_test)
    dd_x_test.iloc[:,2]=dd_x_test.iloc[:,2]+100
    #dd_x_test.iloc[:,0]=0

elif dd_dataset in ('synthetic'):
    dd_x_test=trainX[list(noisy_idx)]
    dd_y_test=trainY[list(noisy_idx)]
    dd_y_test=np.argmax(dd_y_test,axis=1)
    dd_y_test = np.where(dd_y_test==1,0,1)   
    dd_y_test = to_categorical(dd_y_test)

    
#size of OOD data
dd_x_test = dd_x_test[0:t_size]
dd_y_test = dd_y_test[0:t_size]

print("TRZ : ",TRZ, " TEZ : ", TEZ ," SHZ : ", SHZ, "DD : ", dd_x_test.shape[0])
                  

TRZ :  20000  TEZ :  20000  SHZ :  10000 DD :  20000


In [23]:
#shadow train vs shdow test
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = shadow_model(x_shadow, y_shadow, dataset, model_type, num_classes, dim, channel)

#shadow train vs shdow test
noisy_shadow_train_performance, noisy_shadow_test_performance, noisy_attack_data, noisy_x_shadow_train, noisy_y_shadow_train, noisy_x_shadow_test, noisy_y_shadow_test, noisy_shadow_model_init = shadow_model_noisy(x_shadow, y_shadow, dataset, model_type, num_classes, dim, channel)


#train model
n_ind_train_acc, n_ind_test_acc, n_histories, n_BTR, n_BTE, n_ind_test_performance, n_ind_train_performance, n_tm_model, n_target_ood_performance, n_target_dd_performance, pred_status = evaluate_model(x_train, y_train, x_test, y_test,x_test, y_test, dd_x_test, dd_y_test, dataset, model_type, num_classes, dim, channel, init_weights)

i_train_acc.append(n_ind_train_acc)
i_test_acc.append(n_ind_test_acc)

##########################   inlier outlier identification ##########################
ori_test_y_tm = n_tm_model.predict(x_test, verbose=0)
inliers = np.where(np.argmax(y_test,axis=1)== np.argmax(ori_test_y_tm,axis=1))[0]
outliers = np.where(np.argmax(y_test,axis=1)!= np.argmax(ori_test_y_tm,axis=1))[0]
####################################################################################



Shadow Model  0
Shadow Train acc :  100.0 Shadow Test acc :  86.59999966621399
Shadow Model  0
Shadow Train acc :  100.0 Shadow Test acc :  80.26000261306763
Training Acc :  1.0  IND Test Acc :  0.89445  OOD Test Acc :  0.89445  DD Test Acc :  0.0823


In [24]:
print("\n----------Normal random/split member/nonmember data noisy shadow-------------")

print("non-member size : ", len(n_ind_test_performance[0]))
print("member size : ", len(n_ind_test_performance[0]))
         
MIA = black_box_benchmarks(noisy_shadow_train_performance,noisy_shadow_test_performance,
                             n_ind_train_performance,n_ind_test_performance,n_tm_model,num_classes=num_classes)
NOISYMIIND=MIA._mem_inf_benchmarks()



----------Normal random/split member/nonmember data noisy shadow-------------
non-member size :  20000
member size :  20000
For membership inference attack via correctness, the attack acc is 0.553, with train acc 1.000 and test acc 0.894
TP :  20000
TN :  2111
FN :  0
FP :  17889
precision :  0.5278576895668927
recall :  1.0
acc :  0.552775
confidence
TP :  19090
TN :  4049
FN :  910
FP :  15951
precision :  0.5447903884021574
recall :  0.9545
acc :  0.578475
For membership inference attack via confidence, the attack acc is 0.578
entropy
TP :  18867
TN :  3407
FN :  1133
FP :  16593
precision :  0.5320642978003384
recall :  0.94335
acc :  0.55685
For membership inference attack via entropy, the attack acc is 0.557
modified entropy
TP :  19002
TN :  4143
FN :  998
FP :  15857
precision :  0.5451103015003299
recall :  0.9501
acc :  0.578625
For membership inference attack via modified entropy, the attack acc is 0.579


In [25]:
print("\n----------Normal random/split member/nonmember data - Modified entropy MIA-------------")

print("non-member size : ", len(n_ind_test_performance[0]))
print("member size : ", len(n_ind_train_performance[0]))
         
MIA = black_box_benchmarks(n_shadow_train_performance,n_shadow_test_performance,
                             n_ind_train_performance,n_ind_test_performance,n_tm_model,num_classes=num_classes)
NOMIIND=MIA._mem_inf_benchmarks()





----------Normal random/split member/nonmember data - Modified entropy MIA-------------
non-member size :  20000
member size :  20000
For membership inference attack via correctness, the attack acc is 0.553, with train acc 1.000 and test acc 0.894
TP :  20000
TN :  2111
FN :  0
FP :  17889
precision :  0.5278576895668927
recall :  1.0
acc :  0.552775
confidence
TP :  19951
TN :  3500
FN :  49
FP :  16500
precision :  0.5473375216043456
recall :  0.99755
acc :  0.586275
For membership inference attack via confidence, the attack acc is 0.586
entropy
TP :  19765
TN :  2691
FN :  235
FP :  17309
precision :  0.533122943302584
recall :  0.98825
acc :  0.5614
For membership inference attack via entropy, the attack acc is 0.561
modified entropy
TP :  19934
TN :  3525
FN :  66
FP :  16475
precision :  0.5475019912658958
recall :  0.9967
acc :  0.586475
For membership inference attack via modified entropy, the attack acc is 0.586


In [26]:
#correct classification statistics
#train_pred_y
Y_train_pred = n_tm_model.predict_classes(x_train, verbose=0)
Y_train_pred_score = n_tm_model.predict(x_train, verbose=0)
cc_tp = np.where(Y_train_pred==n_ind_train_performance[1])
cc_fn = np.where(Y_train_pred!=n_ind_train_performance[1])


#test_pred_y
Y_test_pred = n_tm_model.predict_classes(x_test, verbose=0)
Y_test_pred_score = n_tm_model.predict(x_test, verbose=0)
cc_tn = np.where(Y_test_pred!=n_ind_test_performance[1])
cc_fp = np.where(Y_test_pred==n_ind_test_performance[1])

cc_tp=cc_tp[0]
cc_fn=cc_fn[0]
cc_tn=cc_tn[0]
cc_fp=cc_fp[0]



In [27]:
# # ################### max confidence score extraction ###########################

ind_train_prob, ind_test_prob = extract_prob(n_ind_train_performance[0],n_ind_test_performance[0])
_, ood_test_prob = extract_prob(n_target_dd_performance[0], n_target_dd_performance[0])


In [28]:
# #####neigbouthood based outliers #####
# #unique class vals
# class_val=np.unique(np.argmax(y_train))

# #membership inference
train_act  = extract_activation(n_tm_model,x_train,y_train,act_layer)
test_act  = extract_activation(n_tm_model,x_test,y_test,act_layer)

outlier_df = pd.concat([train_act,test_act],axis=0)
outlier_df.reset_index(drop=True, inplace=True)

clf = LocalOutlierFactor(n_neighbors=1)
outlier_staus_train = clf.fit_predict(train_act)
outlier_staus_test = clf.fit_predict(test_act)

print("Train : ",Counter(outlier_staus_train))
print("Test : ",Counter(outlier_staus_test))

tr_knnoutlier_idx = np.where(outlier_staus_train==-1)[0]
te_knnoutlier_idx = np.where(outlier_staus_test==-1)[0]

tr_knninlier_idx = np.where(outlier_staus_train==1)[0]
te_knninlier_idx = np.where(outlier_staus_test==1)[0]




Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a94b68d50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a94b299d0>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a94b335d0>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a94b33f50>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a94b38e90>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a94b11390>]
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a94b68d50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a94b299d0>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a94b335d0>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a94b33f50>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a94b38e90>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a94b11390>]
Train :  Counter({1: 17601, -1: 2399})
Test :  Counter({1: 17548, -1: 2452})

In [29]:
# ####### confidence score based outlier ######
# ind_train_prob, ind_test_prob = extract_prob(n_ind_train_performance[0],n_ind_test_performance[0])

# #outliers and inliers
# thresh=0.6
# conf_outlier_indices = np.where(ind_test_prob <= thresh)[0]
# conf_inlier_indices = np.where(ind_test_prob > thresh)[0]

# #get outliers and inliers predictions
# conf_outlier_pred_score = n_ind_test_performance[0][conf_outlier_indices]
# conf_inlier_pred_score = n_ind_test_performance[0][conf_inlier_indices]

# #get outliers and inliers classes
# conf_outlier_class = n_ind_test_performance[1][conf_outlier_indices]
# conf_inlier_class = n_ind_test_performance[1][conf_inlier_indices]

# #concatanate arrays
# conf_outliers=(conf_outlier_pred_score,conf_outlier_class)
# conf_inliers=(conf_inlier_pred_score,conf_inlier_class)

# #Equal datasizes - nonmembers
# Eq_outlier, Eq_inlier = data_extraction(conf_outliers,conf_inliers)

# #Equal datasizes - train data
# Eq_train, _ = data_extraction(n_ind_train_performance,Eq_outlier)

# #Equal datasizes - test data
# _, Eq_test = data_extraction(Eq_train,n_ind_test_performance)

# #Equal datasizes - OOD data
# _, Eq_DD = data_extraction(Eq_train,n_target_dd_performance)



In [30]:
###########correct classification based outliers/inliers############

#get outliers and inliers on correct classification predictions
cc_outlier_pred_score = n_ind_test_performance[0][cc_tn]
cc_inlier_pred_score = n_ind_test_performance[0][cc_fp]

#get outliers and inliers classes
cc_outlier_class = n_ind_test_performance[1][cc_tn]
cc_inlier_class = n_ind_test_performance[1][cc_fp]


#concatanate arrays
cc_outliers=(cc_outlier_pred_score,cc_outlier_class)
cc_inliers=(cc_inlier_pred_score,cc_inlier_class)


# #Equal datasizes - train data
Eq_train, _ = data_extraction(n_ind_train_performance,cc_outliers)

#Equal datasizes - CC inlier data
_, Eq_inlier = data_extraction(Eq_train,cc_inliers)


#Equal datasizes - IC outlier data
_, Eq_outlier = data_extraction(Eq_train,cc_outliers)

# #Equal datasizes - OOD data
_, Eq_DD = data_extraction(Eq_train,n_target_dd_performance)

# #Equal datasizes - test data
_, Eq_test = data_extraction(Eq_train,n_ind_test_performance)


In [31]:
####################### MIA - Classification probability Outliers #######################
print("--------MIA---------------")

train_data_list=[Eq_train, Eq_train]
test_data_list=[Eq_test, Eq_DD]


    
ddconf_iprecision, ddconf_iacc, ddconf_recall, dde_iprecision, dde_iacc, dde_recall, ddme_iprecision, ddme_iacc,ddme_recall, ddcc_precision, ddcc_acc,ddcc_recall = [],[],[],[],[],[],[],[],[],[],[],[]
for i in range(len(train_data_list)):

    print("\n\n")
    if i == 0:
        print("Normal")
    elif i ==1:
        print("OOD")
    print("\n\n")
    
    
    MIA = black_box_benchmarks(n_shadow_train_performance,n_shadow_test_performance,
                             train_data_list[i],test_data_list[i],n_tm_model,num_classes=num_classes)
    ENMEM=MIA._mem_inf_benchmarks()


    
    ddconf_iprecision.append(ENMEM[6])
    ddconf_iacc.append(ENMEM[7])
    ddconf_recall.append(ENMEM[8])
    #print("\n")


    dde_iprecision.append(ENMEM[15])
    dde_iacc.append(ENMEM[16])
    dde_recall.append(ENMEM[17])
    #print("\n")


    ddme_iprecision.append(ENMEM[24])
    ddme_iacc.append(ENMEM[25])
    ddme_recall.append(ENMEM[26])


    ddcc_precision.append(ENMEM[30] )
    ddcc_acc.append(ENMEM[31] )
    ddcc_recall.append(ENMEM[32])
    
    print("\n")


--------MIA---------------



Normal



For membership inference attack via correctness, the attack acc is 0.551, with train acc 1.000 and test acc 0.899
TP :  2111
TN :  214
FN :  0
FP :  1897
precision :  0.5266966067864272
recall :  1.0
acc :  0.5506868782567503
confidence
TP :  2104
TN :  375
FN :  7
FP :  1736
precision :  0.5479166666666667
recall :  0.9966840360018948
acc :  0.5871624822359072
For membership inference attack via confidence, the attack acc is 0.587
entropy
TP :  2086
TN :  289
FN :  25
FP :  1822
precision :  0.5337768679631525
recall :  0.9881572714353387
acc :  0.5625296068214116
For membership inference attack via entropy, the attack acc is 0.563
modified entropy
TP :  2104
TN :  377
FN :  7
FP :  1734
precision :  0.5482021886399167
recall :  0.9966840360018948
acc :  0.5876361913784937
For membership inference attack via modified entropy, the attack acc is 0.588





OOD



For membership inference attack via correctness, the attack acc is 0.957, with train 

In [32]:
# #@@@@
# softmax_prob_train, softmax_prob_test = extract_prob(train_data_list[0][0],test_data_list[0][0])

# _, softmax_prob_DD = extract_prob(train_data_list[1][0],test_data_list[1][0])

# print(np.mean(softmax_prob_train), np.std(softmax_prob_train))
# print(np.mean(softmax_prob_test), np.std(softmax_prob_test))
# print(np.mean(softmax_prob_DD), np.std(softmax_prob_DD))

# #sys.exit("Stop here")

In [33]:
# import seaborn as sns

# fig, ax = plt.subplots(sharey=True)
# sns.distplot( softmax_prob_train , color="orange", hist=False, label="Members",ax=ax)
# sns.distplot( softmax_prob_test , color="dodgerblue", hist=False, kde_kws={'linestyle':'--'}, label="Non-Members (IND)",ax=ax)
# sns.distplot( softmax_prob_DD , color="deeppink", hist=False, kde_kws={'linestyle':':'}, label="Non-Members (OOD/ Noisy)",ax=ax)


# plt.legend();
# plt.xlabel("Max soft-max confidence values")

# # ax2 = plt.axes([0.2, 0.22, .4, .4], facecolor='w')
# # sns.distplot( softmax_prob_train , color="orange", hist=False, label="Members",ax=ax2)
# # sns.distplot( softmax_prob_test , color="dodgerblue", hist=False, kde_kws={'linestyle':'--'}, label="Non-Members (IND)",ax=ax2)
# # sns.distplot( softmax_prob_DD , color="deeppink", hist=False, kde_kws={'linestyle':':'}, label="Non-Members (OOD/ Noisy)",ax=ax2)

# # ax2.set_xlim([0.6,1.04])
# # ax2.set_ylim([0,25])
# # ax2.set(xlabel='',ylabel='')



# plt.tight_layout()
# fig.savefig(img_path+dataset+'_noisy.png', dpi=200) 



In [34]:
# #shadow model

# #set threshold
# shadow_tr_max, shadow_te_max = extract_prob(n_shadow_train_performance[0], n_shadow_test_performance[0])
# thr = thre_setting(shadow_tr_max,shadow_te_max)

# #membership
# tr_max, te_max = extract_prob(Eq_train[0], Eq_test[0])
# new_pred_mem = np.where(tr_max > thr,1,0) 
# new_pred_nmem = np.where(te_max >thr,1,0)

# full_new_pred  = np.concatenate((new_pred_mem,new_pred_nmem))
# full_new_ori = np.concatenate( (np.ones(len(new_pred_mem)), np.zeros(len(new_pred_nmem))))

# tn, fp, fn, tp = confusion_matrix(full_new_ori, full_new_pred).ravel()

# print("TP : ", tp )
# print("TN : ", tn)
# print("FN : ", fn )
# print("FP : ", fp,"\n")                                  

# precision=tp/(tp+fp)
# recall=tp/(tp+fn)
# acc=(tp+tn)/(tp+tn+fn+fp)

# print("precision : ",precision)
# print("recall : ",recall)
# print("acc : ", acc)                                  
# print("\n\n")


In [35]:
#sys.exit()

In [36]:
####################### MIA - Classification probability based methods #######################
print("--------MIA---------------")

#train_data_list=[n_ind_train_performance,Eq_train,Eq_train,Eq_train,Eq_train,Eq_train,Eq_train]
#test_data_list=[n_ind_test_performance,Eq_test,Eq_inlier,Eq_outlier,Eq_DD,Eq_CC_inlier,Eq_CC_outlier]

train_data_list=[Eq_train, Eq_train,     Eq_train]
test_data_list=[ Eq_test,  Eq_inlier, Eq_outlier]



for i in range(len(train_data_list)):

    print("\n\n")
    if i == 0:
        print("Random")
    elif i ==1:
        print("Inlier NM + CC")
    elif i ==2:
        print("Outlier NM + ICC")
    print("\n\n")
    
    
    MIA = black_box_benchmarks(n_shadow_train_performance,n_shadow_test_performance,
                             train_data_list[i],test_data_list[i],n_tm_model,num_classes=num_classes)
    ENMEM=MIA._mem_inf_benchmarks()


    
    conf_iprecision.append(ENMEM[6])
    conf_iacc.append(ENMEM[7])
    conf_recall.append(ENMEM[8])


    e_iprecision.append(ENMEM[15])
    e_iacc.append(ENMEM[16])
    e_recall.append(ENMEM[17])


    me_iprecision.append(ENMEM[24])
    me_iacc.append(ENMEM[25])
    me_recall.append(ENMEM[26])

    per_class_threshold = ENMEM[27]

    true_membership = ENMEM[28]
    pred_membership = ENMEM[29]


    cc_precision.append(ENMEM[30] )
    cc_acc.append(ENMEM[31] )
    cc_recall.append(ENMEM[32])

    if i == 0:
        TP_CONF = ENMEM[0]
        TP_ENTR = ENMEM[9]
        TP_MENTR = ENMEM[18]
        
        TN_CONF = ENMEM[3]
        TN_ENTR = ENMEM[12]
        TN_MENTR = ENMEM[21]
        
        CC_TP = ENMEM[33]
        CC_FN = ENMEM[34]
        CC_TN = ENMEM[35]
        CC_FP = ENMEM[36]
        
        
        
        # # ################### max confidence score extraction ###########################

        softmax_prob_train, softmax_prob_test = extract_prob(train_data_list[0][0],test_data_list[0][0])


        
        

--------MIA---------------



Random



For membership inference attack via correctness, the attack acc is 0.551, with train acc 1.000 and test acc 0.899
TP :  2111
TN :  214
FN :  0
FP :  1897
precision :  0.5266966067864272
recall :  1.0
acc :  0.5506868782567503
confidence
TP :  2104
TN :  375
FN :  7
FP :  1736
precision :  0.5479166666666667
recall :  0.9966840360018948
acc :  0.5871624822359072
For membership inference attack via confidence, the attack acc is 0.587
entropy
TP :  2086
TN :  289
FN :  25
FP :  1822
precision :  0.5337768679631525
recall :  0.9881572714353387
acc :  0.5625296068214116
For membership inference attack via entropy, the attack acc is 0.563
modified entropy
TP :  2104
TN :  377
FN :  7
FP :  1734
precision :  0.5482021886399167
recall :  0.9966840360018948
acc :  0.5876361913784937
For membership inference attack via modified entropy, the attack acc is 0.588



Inlier NM + CC



For membership inference attack via correctness, the attack acc is 0.500, wi

In [37]:
####################### MIA - logloss attack #######################
print("MIA - logloss attack")
logloss_precision=[]
logloss_acc=[]
logloss_recall=[]


### compute loglosss for shadow data
shadow_train_instance_loss  = np.array(log_loss(n_shadow_train_performance[1],n_shadow_train_performance[0]))
shadow_test_instance_loss  = np.array(log_loss(n_shadow_test_performance[1],n_shadow_test_performance[0]))

#compute per class loss threshold values
loss_tresh_list = loss_thre_setting(shadow_train_instance_loss, shadow_test_instance_loss, n_shadow_train_performance[1], n_shadow_test_performance[1])
print("loss_tresh_list : ", loss_tresh_list)

###get predictions for original membership validation data

#prediction
train_instance_pred_tm = n_tm_model.predict(x_train)
test_instance_pred_tm = n_tm_model.predict(x_test)

### compute loglosss for original data
ori_train_instance_loss  = np.array(log_loss(np.argmax(y_train,axis=1),train_instance_pred_tm))
ori_test_instance_loss  = np.array(log_loss(np.argmax(y_test,axis=1),test_instance_pred_tm))



#random membership determination
prec, rec, acc = loss_mia (loss_tresh_list, ori_train_instance_loss, ori_test_instance_loss, np.argmax(train_instance_pred_tm,axis=1), np.argmax(test_instance_pred_tm,axis=1) )
logloss_precision.append(prec)
logloss_recall.append(rec)
logloss_acc.append(acc)


# #concatanate arrays
ll_outliers=(ori_test_instance_loss[outliers],np.argmax(y_test,axis=1)[outliers])
ll_inliers=(ori_test_instance_loss[inliers],np.argmax(y_test,axis=1)[inliers])
ll_train=(ori_train_instance_loss,np.argmax(y_train,axis=1))


# #Equal datasizes - CC inlier data
Eq_TRA_inlier, Eq_TEST_inlier = data_extraction(ll_train,ll_inliers)


# #Equal datasizes - IC outlier data
Eq_TRA_outlier, Eq_TEST_outlier = data_extraction(ll_train,ll_outliers)


#inlier memmbership determination
prec, rec, acc = loss_mia (loss_tresh_list, Eq_TRA_inlier[0], Eq_TEST_inlier[0], Eq_TRA_inlier[1], Eq_TEST_inlier[1])
logloss_precision.append(prec)
logloss_recall.append(rec)
logloss_acc.append(acc)

#outlier
prec, rec, acc = loss_mia (loss_tresh_list, Eq_TRA_outlier[0], Eq_TEST_outlier[0], Eq_TRA_outlier[1], Eq_TEST_outlier[1])
logloss_precision.append(prec)
logloss_recall.append(rec)
logloss_acc.append(acc)



MIA - logloss attack
0
1
2
3
4
5
6
7
8
9
loss_tresh_list :  {0: -0.0, 1: -0.0, 2: -0.0, 3: -0.0, 4: -0.0, 5: -0.0, 6: -0.0, 7: -0.0, 8: -0.0, 9: -0.0}
TP :  0
TN :  20000
FN :  20000
FP :  0
precision :  nan
recall :  0.0
acc :  0.5



TP :  0
TN :  17889
FN :  17889
FP :  0
precision :  nan
recall :  0.0
acc :  0.5



TP :  0
TN :  2111
FN :  2111
FP :  0
precision :  nan
recall :  0.0
acc :  0.5





/home/ec2-user/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/ipykernel/__main__.py:32: RuntimeWarning: invalid value encountered in long_scalars


In [38]:
####################### MIA - Label only transfer attack #######################
print("MIA - Label only transfer attack")
trans_logloss_precision=[]
trans_logloss_acc=[]
trans_logloss_recall=[]

### get target model classifications for shadow data
shadow_pred_y = n_tm_model.predict(x_shadow, verbose=0)



###train modified shadow model
trans_shadow_train_performance, trans_shadow_test_performance, _, trans_x_shadow_train, trans_y_shadow_train, trans_x_shadow_test, trans_y_shadow_test, trans_shadow_model = shadow_model(x_shadow, shadow_pred_y, dataset, model_type, num_classes, dim, channel)

### compute loglosss for shadow data
shadow_train_instance_loss  = np.array(log_loss(trans_shadow_train_performance[1],trans_shadow_train_performance[0]))
shadow_test_instance_loss  = np.array(log_loss(trans_shadow_test_performance[1],trans_shadow_test_performance[0]))

#compute per class loss threshold values
loss_tresh_list = loss_thre_setting(shadow_train_instance_loss, shadow_test_instance_loss, trans_shadow_train_performance[1], trans_shadow_test_performance[1])
print(loss_tresh_list)

###get predictions for original membership validation data

#prediction
train_instance_pred_sm = trans_shadow_model.predict(x_train)
test_instance_pred_sm = trans_shadow_model.predict(x_test)


### compute loglosss for shadow data
ori_train_instance_loss  = np.array(log_loss(np.argmax(y_train,axis=1),train_instance_pred_sm))
ori_test_instance_loss  = np.array(log_loss(np.argmax(y_test,axis=1),test_instance_pred_sm))


#random memmbership determination
prec, rec, acc = loss_mia (loss_tresh_list, ori_train_instance_loss, ori_test_instance_loss, np.argmax(train_instance_pred_sm,axis=1), np.argmax(test_instance_pred_sm,axis=1) )
trans_logloss_precision.append(prec)
trans_logloss_recall.append(rec)
trans_logloss_acc.append(acc)


# #concatanate arrays
trans_outliers=(ori_test_instance_loss[outliers],np.argmax(y_test,axis=1)[outliers])
trans_inliers=(ori_test_instance_loss[inliers],np.argmax(y_test,axis=1)[inliers])
trans_train=(ori_train_instance_loss,np.argmax(y_train,axis=1))

# #train
#shadow_train_loss = (shadow_train_instance_loss,mod_shadow_train_performance[1])


# #Equal datasizes - CC inlier data
Eq_TRA_inlier, Eq_TEST_inlier = data_extraction(trans_train,trans_inliers)


# #Equal datasizes - IC outlier data
Eq_TRA_outlier, Eq_TEST_outlier = data_extraction(trans_train,trans_outliers)


#inlier memmbership determination
prec, rec, acc = loss_mia (loss_tresh_list, Eq_TRA_inlier[0], Eq_TEST_inlier[0], Eq_TRA_inlier[1], Eq_TEST_inlier[1])
trans_logloss_precision.append(prec)
trans_logloss_recall.append(rec)
trans_logloss_acc.append(acc)

#outlier
prec, rec, acc = loss_mia (loss_tresh_list, Eq_TRA_outlier[0], Eq_TEST_outlier[0], Eq_TRA_outlier[1], Eq_TEST_outlier[1])
trans_logloss_precision.append(prec)
trans_logloss_recall.append(rec)
trans_logloss_acc.append(acc)



MIA - Label only transfer attack
Shadow Model  0
Shadow Train acc :  99.3399977684021 Shadow Test acc :  90.03999829292297
0
1
2
3
4
5
6
7
8
9
{0: 1.1920930376163597e-07, 1: 1.1920930376163597e-07, 2: 1.1920930376163597e-07, 3: 1.1920930376163597e-07, 4: 1.1920930376163597e-07, 5: 1.1920930376163597e-07, 6: 1.1920930376163597e-07, 7: 1.1920930376163597e-07, 8: 1.1920930376163597e-07, 9: 1.1920930376163597e-07}
TP :  2347
TN :  17743
FN :  17653
FP :  2257
precision :  0.5097741094700261
recall :  0.11735
acc :  0.50225



TP :  2115
TN :  15632
FN :  15774
FP :  2257
precision :  0.48376029277218663
recall :  0.11822907932248868
acc :  0.4960310805522947



TP :  261
TN :  2111
FN :  1850
FP :  0
precision :  1.0
recall :  0.12363808621506395
acc :  0.561819043107532





In [39]:
####################### MIA - shadow activation attack #######################
# for validation data generate adverserial example

print("MIA - Label only retrainining and activaion attack")

act_precision, act_recall, act_acc = [], [], []


# #prediction-modsm
_, tr_acc = trans_shadow_model.evaluate(x_train, y_train, verbose=0)
_, te_acc = trans_shadow_model.evaluate(x_test, y_test, verbose=0)

print("Mod shadow model - Train Acc: ", tr_acc, "Test Acc : ",te_acc)

#membership inference - valdation set
train_act  = extract_activation(trans_shadow_model,x_train,y_train,act_layer)
test_act  = extract_activation(trans_shadow_model,x_test,y_test,act_layer)

#unique class vals
class_val = np.unique(np.argmax(y_shadow,axis=1))

#MIA
act_df_full = pd.concat([train_act,test_act],axis=0)
act_df_full.reset_index(drop=True, inplace=True)

true_membership = np.concatenate([np.ones(x_train.shape[0]), np.zeros(x_test.shape[0])])


#membership inference - shadow set
shdow_train_act  = extract_activation(trans_shadow_model,trans_x_shadow_train,trans_y_shadow_train,act_layer)
shadow_test_act  = extract_activation(trans_shadow_model,trans_x_shadow_test, trans_y_shadow_test,act_layer)

#MIA
act_df_shadow = pd.concat([shdow_train_act,shadow_test_act],axis=0)
act_df_shadow.reset_index(drop=True, inplace=True)


########## Random ###########

nn=1
pred_membership = activation_mia(act_df_full,class_val,act_layer,nn)

tn, fp, fn, tp = confusion_matrix(true_membership, pred_membership).ravel()

print("TP : ", tp )
print("TN : ", tn)
print("FN : ", fn )
print("FP : ", fp)
        
precision=tp/(tp+fp)
recall=tp/(tp+fn)
acc=(tp+tn)/(tp+tn+fn+fp)

print("precision : ",precision)
print("recall : ",recall)
print("acc : ", acc)
print("\n\n")

act_precision.append(precision)
act_recall.append(recall)
act_acc.append(acc) 





MIA - Label only retrainining and activaion attack
Mod shadow model - Train Acc:  0.88545 Test Acc :  0.8701
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a2025ff10>]
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.

In [40]:
from pyod.models.knn import KNN   # kNN detector
from pyod.models.abod import ABOD
#0: inliers, 1: outliers

def activation_mia_on_shadow(act_df_full,act_df_shadow, class_val,act_layer,nn):
    
    CLASS_IDX, dist_data, pred_mem, outlier, inlier = [], [], [], [], []

    

    outlier_fraction =0.5
    for c_val in class_val:
        print(c_val)

        #validation data activation
        filter_rec_all = act_df_full[(act_df_full['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)
        filter_rec=act_df_full.loc[filter_rec_idx]
        filter_rec=filter_rec.loc[:, filter_rec.columns != 'y']

        #shadow data activation
        shadow_filter_rec_all = act_df_shadow[(act_df_shadow['y'] == c_val)]
        shadow_filter_rec_idx = np.array(shadow_filter_rec_all.index)
        shadow_filter_rec=act_df_shadow.loc[shadow_filter_rec_idx]
        shadow_filter_rec=shadow_filter_rec.loc[:, shadow_filter_rec.columns != 'y']


        clf = KNN(n_neighbors=nn)
        #clf = KNN(contamination=outlier_fraction)
        #clf = ABOD(contamination=outlier_fraction)
   
        clf.fit(shadow_filter_rec)

        # predict raw anomaly score
        scores_pred = clf.decision_function(shadow_filter_rec)*-1
    
        # prediction of a datapoint category outlier or inlier
        filter_outlier = clf.predict(filter_rec)

        filter_outlier_df = pd.DataFrame(filter_outlier)
        filter_outlier_df = filter_outlier_df.set_index(filter_rec_idx)
        filter_outlier_df.columns=['status'] 

        
            
        outlier_df = filter_outlier_df.loc[filter_outlier_df['status'] == 1]
        outlier_idx_to_rec_idx = outlier_df.index.values
        #print(outlier_idx_to_rec_idx)

        inlier_df = filter_outlier_df.loc[filter_outlier_df['status'] == 0]    
        #print(filter_inlier_df)
        inlier_idx_to_rec_idx = inlier_df.index.values
        #print(inlier_idx_to_rec_idx)

        outlier.append(outlier_idx_to_rec_idx)
        inlier.append(inlier_idx_to_rec_idx)
        

    nonmembers = np.array([item for sublist in outlier for item in sublist])
    members = np.array([item for sublist in inlier for item in sublist])
        
    members=np.array(list(members))
    nonmembers=np.array(list(nonmembers))
        
    pred_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
    outlier = np.array([item for sublist in outlier for item in sublist])
    inlier = np.array([item for sublist in inlier for item in sublist])
    
    return pred_membership, outlier, inlier

In [41]:
pred_membership, out_l, in_l = activation_mia_on_shadow(act_df_full, act_df_shadow, class_val,act_layer,10)

tn, fp, fn, tp = confusion_matrix(true_membership, pred_membership).ravel()


print("TP : ", tp )
print("TN : ", tn)
print("FN : ", fn )
print("FP : ", fp)


precision=tp/(tp+fp)
recall=tp/(tp+fn)
acc=(tp+tn)/(tp+tn+fn+fp)

print("precision : ",precision)
print("recall : ",recall)
print("acc : ", acc)
print("\n\n")

0
1
2
3
4
5
6
7
8
9
TP :  20000
TN :  4699
FN :  0
FP :  15301
precision :  0.5665561882099657
recall :  1.0
acc :  0.617475





In [85]:
#data selection

#original data
tr_x = x_train
tr_y = y_train

te_x = x_test
te_y = y_test

v_feat = np.concatenate((tr_x,te_x))
v_class = np.concatenate((tr_y,te_y)) 

#shadow data #relebelled
s_tr_x = trans_x_shadow_train
s_tr_y = trans_y_shadow_train

s_te_x = trans_x_shadow_test
s_te_y = trans_y_shadow_test

s_v_feat = np.concatenate((s_tr_x,s_te_x))
s_v_class = np.concatenate((s_tr_y,s_te_y)) 



In [103]:
def activation_based_MIA (trans_s_model, shadow_feat, shadow_class, val_feat, val_class, unique_label, act_layer, noise, dataset):

    sd = noise 
    if dataset in ('MNIST', 'FMNIST', 'CIFAR10'):
        
        mean = 0.0   # some constant
        noisy_img = val_feat + np.random.normal(mean, sd, val_feat.shape)
        noisy_x = np.clip(noisy_img, 0, 255)

        #distance to original
        dist_list=np.array(distf(val_feat, noisy_x,"img"))

            
    else:
        
        noisy_x = dd_noisy_bin_data(val_feat,sd)

        #distance to original
        dist_list=np.array(distf(val_feat, noisy_x,"-"))

        
    #extract activation for noisy shdow data from shadow model#
    noisy_act_df  = extract_activation(trans_s_model, noisy_x, val_class, act_layer)
    noisy_act_df = pd.DataFrame(noisy_act_df)


    #extract activation for membership validation data from shadow model#
    shadow_act_df  = extract_activation(trans_s_model, shadow_feat, shadow_class, act_layer)
    shadow_act_df = pd.DataFrame(shadow_act_df)
    
    sz = int(val_feat.shape[0]/2)
    true_membership = np.concatenate([np.ones(sz), np.zeros(sz)])
    
    pred_membership, _, _ = activation_mia_on_shadow(noisy_act_df, shadow_act_df, class_val, act_layer, nn)
   
    tn, fp, fn, tp = confusion_matrix(true_membership, pred_membership).ravel()

    print("TP : ", tp )
    print("TN : ", tn)
    print("FN : ", fn )
    print("FP : ", fp)

    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    acc=(tp+tn)/(tp+tn+fn+fp)

    print("precision : ",precision)
    print("recall : ",recall)
    print("acc : ", acc)
    print("\n\n")

#     shadow_act_precision.append(precision)
#     shadow_act_recall.append(recall)
#     shadow_act_acc.append(acc)     
    
     
    return acc, precision, recall 
        
        
        

In [104]:
##### Find noise parameter

shadow_act_precision, shadow_act_recall, shadow_act_acc =[], [], [] 


MEMT, NONMEMT = [], []
img_noise_range=[0.01, 0.1, 0.15, 0.2, 0.3]
bin_noise_range=[0.1, 0.2, 0.3, 0.4, 0.5]

nn=10


#unique class vals
class_val = np.unique(np.argmax(s_v_class,axis=1))

for t in range(5):
    
    print(t)    
    
    
        
    if dataset in ('MNIST', 'FMNIST', 'CIFAR10'):
        noise = img_noise_range[t]
            
    else:
        noise = bin_noise_range[t]
        
    
    acc, precision, recall  = activation_based_MIA (trans_shadow_model, s_tr_x, s_tr_y, s_v_feat, s_v_class, class_val, act_layer,noise, dataset)
    

    shadow_act_precision.append(precision)
    shadow_act_recall.append(recall)
    shadow_act_acc.append(acc) 

0
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a2025ff10>]
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a2025ff10>]
0
1
2
3
4
5
6
7
8
9
TP :  5000
TN :  1131
FN :  0
FP :  3869
precision :  

/home/ec2-user/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/ipykernel/__main__.py:43: RuntimeWarning: invalid value encountered in long_scalars


In [112]:
#####best noise value

best_sd_list = np.add(np.array(shadow_act_precision)*0.5, np.array(shadow_act_acc)*0.5)
best_sd_pos = np.nanargmax(best_sd_list)


img_noise_range=[0.01, 0.1, 0.15, 0.2, 0.3]
bin_noise_range=[0.1, 0.2, 0.3, 0.4, 0.5]

if dataset in ('MNIST', 'FMNIST', 'CIFAR10'):
    noise = img_noise_range[best_sd_pos]
else:
    noise = bin_noise_range[best_sd_pos]



In [113]:
noise

0.15

In [114]:
MEMT, NONMEMT = [], []
nn=10

new_act_precision, new_act_recall, new_act_acc =[], [], []

acc, precision, recall  = activation_based_MIA (trans_shadow_model, s_tr_x, s_tr_y, v_feat, v_class, class_val, act_layer,noise, dataset)
    


new_act_precision.append(precision)
new_act_recall.append(recall)
new_act_acc.append(acc) 

Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a2025ff10>]
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a2025ff10>]
0
1
2
3
4
5
6
7
8
9
TP :  16923
TN :  20000
FN :  3077
FP :  0
precision :  

In [43]:
# ##### Find noise parameter

# shadow_act_precision, shadow_act_recall, shadow_act_acc =[], [], [] 


# MEMT, NONMEMT = [], []
# img_noise_range=[0.01, 0.1, 0.15, 0.2, 0.3]
# bin_noise_range=[0.1, 0.2, 0.3, 0.4, 0.5]

# nn=10

# for t in range(5):
    
#     print(t)    
    
#     #activation_based_MIA (s_model,feat,labels,unique_label,alact_layer,shadow_status)
    
    
    
#     if dataset in ('MNIST', 'FMNIST', 'CIFAR10'):
        
#         mean = 0.0   # some constant
#         sd = img_noise_range[t]   # some constant (standard deviation)
#         noisy_img = s_v_feat + np.random.normal(mean, sd, s_v_feat.shape)
#         noisy_x = np.clip(noisy_img, 0, 255)

#         #distance to original
#         dist_list=np.array(distf(s_v_feat, noisy_x,"img"))
       
            
#     else:
#         sd = bin_noise_range[t]
#         noisy_x = dd_noisy_bin_data(s_v_feat,sd)
          
#         #distance to original
#         dist_list=np.array(distf(s_v_feat, noisy_x,"-"))
        
        
#     #membership inference - valdation set
#     shadow_ori_train_act  = extract_activation(trans_shadow_model,noisy_x,s_v_class,act_layer)

    
    
#     #unique class vals
#     class_val = np.unique(np.argmax(s_v_class,axis=1))

#     #MIA
#     shadow_act_df_noisy = pd.DataFrame(shadow_ori_train_act)
#     shadow_act_df_noisy.reset_index(drop=True, inplace=True)

#     true_membership = np.concatenate([np.ones(s_tr_x.shape[0]), np.zeros(s_te_x.shape[0])])


#     #membership inference - shadow set
#     shadow_train_act  = extract_activation(trans_shadow_model,trans_x_shadow_train,trans_y_shadow_train,act_layer)
#     shadow_test_act  = extract_activation(trans_shadow_model,trans_x_shadow_test, trans_y_shadow_test,act_layer)

#     #MIA
#     ori_act_df_shadow = pd.concat([shadow_train_act,shadow_test_act],axis=0)
#     ori_act_df_shadow.reset_index(drop=True, inplace=True)


#     ########## Random ###########

    
    
#     pred_membership, out_l, in_l = activation_mia_on_shadow(shadow_ori_train_act, ori_act_df_shadow, class_val, act_layer, nn)
#     MEMT.append(in_l)
#     NONMEMT.append(out_l)
    
#     tn, fp, fn, tp = confusion_matrix(true_membership, pred_membership).ravel()

#     print("TP : ", tp )
#     print("TN : ", tn)
#     print("FN : ", fn )
#     print("FP : ", fp)

#     precision=tp/(tp+fp)
#     recall=tp/(tp+fn)
#     acc=(tp+tn)/(tp+tn+fn+fp)

#     print("precision : ",precision)
#     print("recall : ",recall)
#     print("acc : ", acc)
#     print("\n\n")

#     shadow_act_precision.append(precision)
#     shadow_act_recall.append(recall)
#     shadow_act_acc.append(acc) 

0
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a2025ff10>]
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a2025ff10>]
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object

/home/ec2-user/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/ipykernel/__main__.py:74: RuntimeWarning: invalid value encountered in long_scalars


In [87]:
# MEMT, NONMEMT = [], []
# max_t=1
# nn=10
# for t in range(max_t):
#     print(t)    
#     if dataset in ('MNIST', 'FMNIST', 'CIFAR10'):

#         mean = 0.0   # some constant
#         sd =0.2   # some constant (standard deviation)
#         noisy_img = v_feat + np.random.normal(mean, sd, v_feat.shape)
#         noisy_x = np.clip(noisy_img, 0, 255)

#         #distance to original
#         dist_list=np.array(distf(v_feat, noisy_x,"img"))
       
            
#     else:
#         noisy_x = dd_noisy_bin_data(v_feat,sd)
          
#         #distance to original
#         dist_list=np.array(distf(v_feat, noisy_x,"-"))
        
        
#     #membership inference - valdation set
#     train_act  = extract_activation(trans_shadow_model,noisy_x,v_class,act_layer)

    
    
#     #unique class vals
#     class_val = np.unique(np.argmax(v_class,axis=1))

#     #MIA
#     act_df_noisy = pd.DataFrame(train_act)
#     act_df_noisy.reset_index(drop=True, inplace=True)

#     true_membership = np.concatenate([np.ones(tr_x.shape[0]), np.zeros(te_x.shape[0])])


#     #membership inference - shadow set
#     shadow_train_act  = extract_activation(trans_shadow_model,trans_x_shadow_train,trans_y_shadow_train,act_layer)
#     shadow_test_act  = extract_activation(trans_shadow_model,trans_x_shadow_test, trans_y_shadow_test,act_layer)

#     #MIA
#     act_df_ori_shadow = pd.concat([shadow_train_act,shadow_test_act],axis=0)
#     act_df_ori_shadow.reset_index(drop=True, inplace=True)


#     ########## Random ###########

    
    
#     pred_membership, out_l, in_l = activation_mia_on_shadow(act_df_noisy, act_df_ori_shadow, class_val, act_layer, nn)
#     MEMT.append(in_l)
#     NONMEMT.append(out_l)
    
#     tn, fp, fn, tp = confusion_matrix(true_membership, pred_membership).ravel()

#     print("TP : ", tp )
#     print("TN : ", tn)
#     print("FN : ", fn )
#     print("FP : ", fp)

#     precision=tp/(tp+fp)
#     recall=tp/(tp+fn)
#     acc=(tp+tn)/(tp+tn+fn+fp)

#     print("precision : ",precision)
#     print("recall : ",recall)
#     print("acc : ", acc)
#     print("\n\n")

#     act_precision.append(precision)
#     act_recall.append(recall)
#     act_acc.append(acc) 

0
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a2025ff10>]
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3a2038de50>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f3a20312d90>, <tensorflow.python.keras.layers.core.Flatten object at 0x7f3a20300090>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a202ed510>, <tensorflow.python.keras.layers.core.Activation object at 0x7f3a202d3310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f3a2025ff10>]
Layer list : 
[<tensorflow.python.keras.layers.convolutional.Conv2D object

In [45]:
# from pyod.models.knn import KNN   # kNN detector
# from pyod.models.abod import ABOD
# #0: inliers, 1: outliers

# def activation_mia_on_shadow(act_df_full, act_df_shadow, class_val, act_layer, nn):
    
#     CLASS_IDX, dist_data, pred_mem, outlier, inlier = [], [], [], [], []

    

#     outlier_fraction =0.5
#     for c_val in class_val:
#         print(c_val)

#         #validation data activation
#         filter_rec_all = act_df_full[(act_df_full['y'] == c_val)]
#         filter_rec_idx = np.array(filter_rec_all.index)
#         filter_rec=act_df_full.loc[filter_rec_idx]
#         filter_rec=filter_rec.loc[:, filter_rec.columns != 'y']

#         #shadow data activation
#         shadow_filter_rec_all = act_df_shadow[(act_df_shadow['y'] == c_val)]
#         shadow_filter_rec_idx = np.array(shadow_filter_rec_all.index)
#         shadow_filter_rec=act_df_shadow.loc[shadow_filter_rec_idx]
#         shadow_filter_rec=shadow_filter_rec.loc[:, shadow_filter_rec.columns != 'y']


#         #clf = KNN(n_neighbors=nn)
#         clf = KNN(contamination=outlier_fraction)
#         #clf = ABOD(contamination=outlier_fraction)
   
#         clf.fit(shadow_filter_rec)

#         # predict raw anomaly score
#         scores_pred = clf.decision_function(shadow_filter_rec)*-1

#         # prediction of a datapoint category outlier or inlier
#         filter_outlier = clf.predict(filter_rec)

#         filter_outlier_df = pd.DataFrame(filter_outlier)
#         filter_outlier_df = filter_outlier_df.set_index(filter_rec_idx)
#         filter_outlier_df.columns=['status'] 

        
#         #nonmembers    
#         outlier_df = filter_outlier_df.loc[filter_outlier_df['status'] == 1]
#         outlier_idx_to_rec_idx = outlier_df.index.values
#         #print(outlier_idx_to_rec_idx)

#         #members
#         inlier_df = filter_outlier_df.loc[filter_outlier_df['status'] == 0]    
#         #print(filter_inlier_df)
#         inlier_idx_to_rec_idx = inlier_df.index.values
#         #print(inlier_idx_to_rec_idx)

#         outlier.append(outlier_idx_to_rec_idx)
#         inlier.append(inlier_idx_to_rec_idx)
        

#     nonmembers = np.array([item for sublist in outlier for item in sublist])
#     members = np.array([item for sublist in inlier for item in sublist])
        
#     members=np.array(list(members))
#     nonmembers=np.array(list(nonmembers))
        
#     pred_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
#     outlier = np.array([item for sublist in outlier for item in sublist])
#     inlier = np.array([item for sublist in inlier for item in sublist])
    
#     return pred_membership, outlier, inlier

In [46]:
# from pyod.models.knn import KNN   # kNN detector
# from pyod.models.abod import ABOD
# #0: inliers, 1: outliers

# def activation_mia_on_shadow(act_df_full, act_df_shadow, class_val, act_layer, nn):
    
#     CLASS_IDX, dist_data, pred_mem, outlier, inlier = [], [], [], [], []

    

#     outlier_fraction =0.5
#     for c_val in class_val:
#         print(c_val)

#         #validation data activation
#         filter_rec_all = act_df_full[(act_df_full['y'] == c_val)]
#         filter_rec_idx = np.array(filter_rec_all.index)
#         filter_rec=act_df_full.loc[filter_rec_idx]
#         filter_rec=filter_rec.loc[:, filter_rec.columns != 'y']

#         #shadow data activation
#         shadow_filter_rec_all = act_df_shadow[(act_df_shadow['y'] == c_val)]
#         shadow_filter_rec_idx = np.array(shadow_filter_rec_all.index)
#         shadow_filter_rec=act_df_shadow.loc[shadow_filter_rec_idx]
#         shadow_filter_rec=shadow_filter_rec.loc[:, shadow_filter_rec.columns != 'y']


#         #clf = KNN(n_neighbors=nn)
#         clf = KNN(contamination=outlier_fraction)
#         #clf = ABOD(contamination=outlier_fraction)
   
#         clf.fit(shadow_filter_rec)

#         # predict raw anomaly score
#         scores_pred = clf.decision_function(shadow_filter_rec)*-1

#         # prediction of a datapoint category outlier or inlier
#         filter_outlier = clf.predict(filter_rec)

#         filter_outlier_df = pd.DataFrame(filter_outlier)
#         filter_outlier_df = filter_outlier_df.set_index(filter_rec_idx)
#         filter_outlier_df.columns=['status'] 

        
#         #nonmembers    
#         outlier_df = filter_outlier_df.loc[filter_outlier_df['status'] == 1]
#         outlier_idx_to_rec_idx = outlier_df.index.values
#         #print(outlier_idx_to_rec_idx)

#         #members
#         inlier_df = filter_outlier_df.loc[filter_outlier_df['status'] == 0]    
#         #print(filter_inlier_df)
#         inlier_idx_to_rec_idx = inlier_df.index.values
#         #print(inlier_idx_to_rec_idx)

#         outlier.append(outlier_idx_to_rec_idx)
#         inlier.append(inlier_idx_to_rec_idx)
        

#     nonmembers = np.array([item for sublist in outlier for item in sublist])
#     members = np.array([item for sublist in inlier for item in sublist])
        
#     members=np.array(list(members))
#     nonmembers=np.array(list(nonmembers))
        
#     pred_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
#     outlier = np.array([item for sublist in outlier for item in sublist])
#     inlier = np.array([item for sublist in inlier for item in sublist])
    
#     return pred_membership, outlier, inlier


# #change the size of shadow model


# ####################### MIA - Label only transfer attack #######################

# sh_act_precision, sh_act_recall, sh_act_acc = [], [], []

# #print("MIA - Label only transfer attack")

# #different shadow sizes
# shadow_size = np.arange(200,x_shadow.shape[0],500)
# shadow_size = np.concatenate((shadow_size, [x_shadow.shape[0]]))
# shadow_size = [x_shadow.shape[0],x_shadow.shape[0]]

# ### get target model classifications for shadow data
# shadow_pred_y = n_tm_model.predict(x_shadow, verbose=0)

# sh_idx=list(range(0,x_shadow.shape[0]))

# tr_x = x_train[0:10]
# tr_y = y_train[0:10]

# te_x = x_test[0:200]
# te_y = y_test[0:200]

# MEM, NMEM = [], []


# for sz in shadow_size:
#     print(sz)
    
#     #ranomly select sz number of samples
#     rand_sam = random.sample(sh_idx,sz)
    
#     #select random number of samples
#     s_x = x_shadow[np.array(rand_sam)]
#     s_y = shadow_pred_y[np.array(rand_sam)]
#     s_y = y_shadow[np.array(rand_sam)]
         
    
    
#     ###train modified shadow model
#     trans_shadow_train_performance, trans_shadow_test_performance,_, sx_train, sy_train, sx_test, sy_test, trans_shadow_model = shadow_model(s_x,s_y, dataset, model_type, num_classes, dim, channel)

#     act_precision, act_recall, act_acc = [], [], []


#     # #prediction-modsm
#     _, tr_acc = trans_shadow_model.evaluate(tr_x, tr_y, verbose=0)
#     _, te_acc = trans_shadow_model.evaluate(te_x, te_y, verbose=0)

#     print("Mod shadow model - Train Acc: ", tr_acc, "Test Acc : ",te_acc)

#     #membership inference - valdation set
#     train_act  = extract_activation(trans_shadow_model,tr_x,tr_y,act_layer)
#     test_act  = extract_activation(trans_shadow_model,te_x,te_y,act_layer)

#     #unique class vals
#     class_val = np.unique(np.argmax(s_y,axis=1))

#     #MIA
#     act_df_full = pd.concat([train_act,test_act],axis=0)
#     act_df_full.reset_index(drop=True, inplace=True)

#     true_membership = np.concatenate([np.ones(tr_x.shape[0]), np.zeros(te_x.shape[0])])


#     #membership inference - shadow set
#     shdow_train_act  = extract_activation(trans_shadow_model,sx_train,sy_train,act_layer)
#     shadow_test_act  = extract_activation(trans_shadow_model,sx_test, sy_test,act_layer)

#     #MIA
#     act_df_shadow = pd.concat([shdow_train_act,shadow_test_act],axis=0)
#     act_df_shadow.reset_index(drop=True, inplace=True)


#     ########## Random ###########

    
#     pred_membership, nonmem, mem = activation_mia_on_shadow(act_df_full, act_df_shadow, class_val, act_layer, nn)
#     MEM.append(mem)
#     NMEM.append(nonmem)

#     tn, fp, fn, tp = confusion_matrix(true_membership, pred_membership).ravel()

#     print("TP : ", tp )
#     print("TN : ", tn)
#     print("FN : ", fn )
#     print("FP : ", fp)

#     precision=tp/(tp+fp)
#     recall=tp/(tp+fn)
#     acc=(tp+tn)/(tp+tn+fn+fp)

#     print("precision : ",precision)
#     print("recall : ",recall)
#     print("acc : ", acc)
#     print("\n\n")

#     sh_act_precision.append(precision)
#     sh_act_recall.append(recall)
#     sh_act_acc.append(acc) 


In [47]:
# #change the size of shadow model


# ####################### MIA - Label only transfer attack #######################

# sh_act_precision, sh_act_recall, sh_act_acc = [], [], []

# #print("MIA - Label only transfer attack")

# #different shadow sizes
# shadow_size = np.arange(200,x_shadow.shape[0],500)
# shadow_size = np.concatenate((shadow_size, [x_shadow.shape[0]]))


# ### get target model classifications for shadow data
# shadow_pred_y = n_tm_model.predict(x_shadow, verbose=0)

# sh_idx=list(range(0,x_shadow.shape[0]))

# tr_x = x_train[0:200]
# tr_y = y_train[0:200]

# te_x = x_test[0:200]
# te_y = y_test[0:200]


# MEM, NMEM = [], []


# #for sz in shadow_size:
# for sz in range(0,50):

#     print(sz)
    
#     #ranomly select sz number of samples
#     rand_sam = random.sample(sh_idx,200)
    
#     #select random number of samples
#     s_x = x_shadow[np.array(rand_sam)]
#     s_y = shadow_pred_y[np.array(rand_sam)]
#     s_y = y_shadow[np.array(rand_sam)]
         
    
    
#     ###train modified shadow model
#     trans_shadow_train_performance, trans_shadow_test_performance,_, sx_train, sy_train, sx_test, sy_test, trans_shadow_model = shadow_model(s_x,s_y, dataset, model_type, num_classes, dim, channel)

#     act_precision, act_recall, act_acc = [], [], []


#     # #prediction-modsm
#     _, tr_acc = trans_shadow_model.evaluate(tr_x, tr_y, verbose=0)
#     _, te_acc = trans_shadow_model.evaluate(te_x, te_y, verbose=0)

#     print("Mod shadow model - Train Acc: ", tr_acc, "Test Acc : ",te_acc)

#     #membership inference - valdation set
#     train_act  = extract_activation(trans_shadow_model,tr_x,tr_y,act_layer)
#     test_act  = extract_activation(trans_shadow_model,te_x,te_y,act_layer)

#     #unique class vals
#     class_val = np.unique(np.argmax(s_y,axis=1))

#     #MIA
#     act_df_full = pd.concat([train_act,test_act],axis=0)
#     act_df_full.reset_index(drop=True, inplace=True)

#     true_membership = np.concatenate([np.ones(tr_x.shape[0]), np.zeros(te_x.shape[0])])


#     #membership inference - shadow set
#     shdow_train_act  = extract_activation(trans_shadow_model,sx_train,sy_train,act_layer)
#     shadow_test_act  = extract_activation(trans_shadow_model,sx_test, sy_test,act_layer)

#     #MIA
#     act_df_shadow = pd.concat([shdow_train_act,shadow_test_act],axis=0)
#     act_df_shadow.reset_index(drop=True, inplace=True)


#     ########## Random ###########

    
#     pred_membership, nonmem, mem = activation_mia_on_shadow(act_df_full, act_df_shadow, class_val, act_layer, nn)
#     MEM.append(mem)
#     NMEM.append(nonmem)

#     tn, fp, fn, tp = confusion_matrix(true_membership, pred_membership).ravel()

#     print("TP : ", tp )
#     print("TN : ", tn)
#     print("FN : ", fn )
#     print("FP : ", fp)

#     precision=tp/(tp+fp)
#     recall=tp/(tp+fn)
#     acc=(tp+tn)/(tp+tn+fn+fp)

#     print("precision : ",precision)
#     print("recall : ",recall)
#     print("acc : ", acc)
#     print("\n\n")

#     sh_act_precision.append(precision)
#     sh_act_recall.append(recall)
#     sh_act_acc.append(acc) 


In [48]:
# plt.plot(sh_act_precision,marker="o")
# plt.plot(sh_act_acc,marker="o")

In [49]:
# pred_train_y_tm = np.argmax(n_tm_model.predict(x_train, verbose=0),axis=1)
# pred_test_y_tm = np.argmax(n_tm_model.predict(x_test, verbose=0),axis=1)
# all_pred = np.concatenate((pred_train_y_tm,pred_test_y_tm))

# ori_train_y_tm = np.argmax(y_train,axis=1)
# ori_test_y_tm = np.argmax(y_test,axis=1)
# all_ori = np.concatenate((ori_train_y_tm,ori_test_y_tm))

# all_correct_idx = np.where(all_pred==all_ori)[0]
# all_incorrect_idx = np.where(all_pred!=all_ori)[0]

# #pred non-members with incorrect classifications
# ACT_IC_TN = len(intersection(out_l,all_incorrect_idx ))/ len(all_incorrect_idx)
# ACT_IC_TP = len(intersection(in_l,all_correct_idx ))/ len(all_correct_idx)


# print("TN : ", ACT_IC_TN)
# print("TP :", ACT_IC_TP)

In [50]:
# part_train = np.concatenate((tr_x,te_x))


# global_activation = [K.function([n_tm_model.input], 
#                               [layer.output])([np.array(part_train), 1])
#                               for layer in n_tm_model.layers]


# #extract activations for each layer
# list_point=[]
# for ly in range(len(global_activation)-1):
#     #print(ly)
#     data_point=[]
#     for int in range(part_train.shape[0]):
#         #print(int)
#         arr1=global_activation[ly][0][int]
#         data_point.append(arr1.ravel())
#     list_point.append(data_point)
    
    
# #organize activations per data instance

# activation_array = []
# for ins in range(0,part_train.shape[0]):
#     print(ins)
#     activation_array_tmp=[]
#     for ly in range(len(list_point)):
#         dz = list_point[ly][ins]
#         activation_array_tmp.append(dz)
#     activation_array_tmp = np.array(activation_array_tmp)
#     activation_array_tmp = np.array([item for sublist in activation_array_tmp for item in sublist])
#     activation_array.append(activation_array_tmp.ravel())

# # mod_act = []
# # for ins in range(0,part_train.shape[0]):
# #     mod_act.append(np.where(activation_array[ins]>0, 1, 0))


# # def n_activation(model,trainX,trainY,act_layer):
# #     activation_list=[]
 
# #     print("Layer list : ")
# #     print(model.layers)
    
# #     global_activation = [K.function([model.input], 
# #                               [layer.output])([np.array(trainX), 1])
# #                               for layer in model.layers]

    
# #     #print(pd.DataFrame(global_activation[act_layer][0]))
# #     #print(pd.DataFrame(global_activation[act_layer]))
    
# #     global_Df = pd.DataFrame(global_activation[act_layer][0])
    

    
# #     train_ix = list(range(trainY.shape[0]))
# #     global_Df['y'] = np.argmax(trainY,axis=1)
# #     class_val = np.unique(np.argmax(trainY,axis=1))
# #     train_df = global_Df
    
# #     return train_df
